despacho:ajedrez,arte,bienestar,infantil y juvenil y fabulas,filosofia,historia,novela

buhardilla:ciencia,geografia,diccionarios

salon:religion,mitologia,arte(pintura)

In [11]:
import json
import unicodedata
import string

class Biblioteca:
    def __init__(self):
        self.catalogo = {}      # Diccionario de secciones -> subsecciones -> lista de libros
        self.contadores = {}    # Contadores para libros sin subsección

    def agregar_seccion(self, codigo, nombre, subsecciones=None):
        """Agrega una sección con subsecciones opcionales."""
        if subsecciones is None:
            subsecciones = []
        self.catalogo[codigo] = {"nombre": nombre, "subsecciones": {}, "libros": [], "subcontadores": {}}
        self.contadores[codigo] = 0
        for sub in subsecciones:
            self.catalogo[codigo]["subsecciones"][sub] = []

    def _normalizar(self, texto):
        texto = (texto or "").lower().strip()
        texto = unicodedata.normalize("NFD", texto)
        texto = "".join(c for c in texto if unicodedata.category(c) != "Mn")
        texto = "".join(c for c in texto if c not in string.punctuation)
        return texto

    def _generar_subcodigo(self, subseccion):
        palabras = subseccion.split()
        if len(palabras) == 1:
            return palabras[0][:4].upper()
        else:
            return ''.join(p[:2].upper() for p in palabras)

    def _siguiente_numero(self, usados):
        """Devuelve el primer número libre a partir de la lista de referencias usadas."""
        n = 1
        usados = sorted(usados)
        for u in usados:
            if u == n:
                n += 1
            elif u > n:
                break
        return n

    def agregar_libro(self, codigo, titulo, autor, subseccion=None, numero=None):
        """Agrega un libro evitando duplicados. Usa huecos libres en la numeración."""
        ntitulo = self._normalizar(titulo)
        nautor = self._normalizar(autor)

        # Revisar duplicados en sección principal
        for libro in self.catalogo[codigo]["libros"]:
            if self._normalizar(libro["titulo"]) == ntitulo and self._normalizar(libro["autor"]) == nautor:
                print(f"El libro '{titulo}' de {autor} ya existe en la sección {self.catalogo[codigo]['nombre']}.")
                return libro["referencia"]

        # Revisar duplicados en subsecciones
        for sub, lista in self.catalogo[codigo]["subsecciones"].items():
            for libro in lista:
                if self._normalizar(libro["titulo"]) == ntitulo and self._normalizar(libro["autor"]) == nautor:
                    print(f"El libro '{titulo}' de {autor} ya existe en la subsección {sub}.")
                    return libro["referencia"]

        # Asignar número
        if subseccion:
            if subseccion not in self.catalogo[codigo]["subcontadores"]:
                self.catalogo[codigo]["subcontadores"][subseccion] = 0

            usados = [int(l["referencia"].split(".")[-1]) for l in self.catalogo[codigo]["subsecciones"].get(subseccion, [])]
            if numero is None:
                numero = self._siguiente_numero(usados)
            self.catalogo[codigo]["subcontadores"][subseccion] = max(self.catalogo[codigo]["subcontadores"][subseccion], numero)

            subcodigo = self._generar_subcodigo(subseccion)
            referencia = f"{codigo}.{subcodigo}.{numero}"
        else:
            usados = [int(l["referencia"].split(".")[-1]) for l in self.catalogo[codigo]["libros"]]
            if numero is None:
                numero = self._siguiente_numero(usados)
            self.contadores[codigo] = max(self.contadores.get(codigo, 0), numero)
            referencia = f"{codigo}.{numero}"

        libro = {"referencia": referencia, "titulo": titulo, "autor": autor}

        if subseccion and subseccion in self.catalogo[codigo]["subsecciones"]:
            self.catalogo[codigo]["subsecciones"][subseccion].append(libro)
        else:
            self.catalogo[codigo]["libros"].append(libro)

        return referencia

    def cambiar_numero_libro(self, codigo, titulo, autor, nuevo_numero, subseccion=None):
        """Cambia el número de referencia de un libro existente."""
        libros = self.catalogo[codigo]["libros"] if subseccion is None else self.catalogo[codigo]["subsecciones"].get(subseccion, [])
        subcodigo = self._generar_subcodigo(subseccion) if subseccion else None

        for libro in libros:
            if self._normalizar(libro["titulo"]) == self._normalizar(titulo) and self._normalizar(libro["autor"]) == self._normalizar(autor):
                if subcodigo:
                    libro["referencia"] = f"{codigo}.{subcodigo}.{nuevo_numero}"
                    self.catalogo[codigo]["subcontadores"][subseccion] = max(self.catalogo[codigo]["subcontadores"].get(subseccion, 0), nuevo_numero)
                else:
                    libro["referencia"] = f"{codigo}.{nuevo_numero}"
                    self.contadores[codigo] = max(self.contadores[codigo], nuevo_numero)
                print(f"Referencia actualizada: {libro['referencia']}")
                return libro["referencia"]

        print("No se encontró el libro para cambiar el número.")
        return None

    def buscar_libros(self, seccion=None, subseccion=None, autor=None, titulo=None, todos=False):
        """
        Búsqueda flexible por sección, subsección, autor o título.
        Devuelve resultados ordenados numéricamente por referencia.
        """
        encontrados = []

        n_subbus = self._normalizar(subseccion) if subseccion else None
        n_autor = self._normalizar(autor) if autor else None
        n_titulo = self._normalizar(titulo) if titulo else None

        for cod, sec in self.catalogo.items():
            if seccion and cod != seccion and not todos:
                continue

            if n_subbus:
                for sub_name, lista in sec["subsecciones"].items():
                    if self._normalizar(sub_name) != n_subbus:
                        continue
                    for libro in lista:
                        if (not n_autor or n_autor in self._normalizar(libro["autor"])) and \
                           (not n_titulo or n_titulo in self._normalizar(libro["titulo"])):
                            encontrados.append(libro)
            else:
                for libro in sec["libros"]:
                    if (not n_autor or n_autor in self._normalizar(libro["autor"])) and \
                       (not n_titulo or n_titulo in self._normalizar(libro["titulo"])):
                        encontrados.append(libro)
                for sub_name, lista in sec["subsecciones"].items():
                    for libro in lista:
                        if (not n_autor or n_autor in self._normalizar(libro["autor"])) and \
                           (not n_titulo or n_titulo in self._normalizar(libro["titulo"])):
                            encontrados.append(libro)

        # ORDENAR por referencia numéricamente
        def clave_ref(libro):
            partes = libro["referencia"].split(".")
            clave = []
            for p in partes:
                if p.isdigit():
                    clave.append(int(p))
                else:
                    clave.append(p)
            return clave

        encontrados.sort(key=clave_ref)

        if not encontrados:
            print("No se encontró ningún libro que coincida con los criterios.")
        else:
            for libro in encontrados:
                print(f"{libro['referencia']} - {libro['titulo']} ({libro['autor']})")

        return encontrados

    def eliminar_libro(self, codigo, titulo, autor, subseccion=None):
        """Elimina un libro de la sección o subsección indicada."""
        ntitulo = self._normalizar(titulo)
        nautor = self._normalizar(autor)

        if subseccion is None:
            for libro in self.catalogo[codigo]["libros"]:
                if self._normalizar(libro["titulo"]) == ntitulo and self._normalizar(libro["autor"]) == nautor:
                    self.catalogo[codigo]["libros"].remove(libro)
                    print(f"Libro eliminado: {libro['referencia']} - {libro['titulo']} ({libro['autor']})")
                    return True
        else:
            if subseccion in self.catalogo[codigo]["subsecciones"]:
                for libro in self.catalogo[codigo]["subsecciones"][subseccion]:
                    if self._normalizar(libro["titulo"]) == ntitulo and self._normalizar(libro["autor"]) == nautor:
                        self.catalogo[codigo]["subsecciones"][subseccion].remove(libro)
                        print(f"Libro eliminado: {libro['referencia']} - {libro['titulo']} ({libro['autor']})")
                        return True

        print("No se encontró el libro para eliminar.")
        return False

    def guardar(self, archivo="biblioteca.json"):
        with open(archivo, "w", encoding="utf-8") as f:
            json.dump(self.catalogo, f, ensure_ascii=False, indent=4)

    def cargar(self, archivo="biblioteca.json"):
        with open(archivo, "r", encoding="utf-8") as f:
            self.catalogo = json.load(f)


In [ ]:
# Se crean las secciones con su abreviatura de referencia asociada:

biblio = Biblioteca()

biblio.agregar_seccion("AJED", "Ajedrez")

biblio.agregar_seccion("ARTE", "Arte", ["Cine", "Museos", "Música", "Pintura", "Poesía", "Teatro"]) 

biblio.agregar_seccion("BIEN", "Bienestar")

biblio.agregar_seccion("CIEN", "Ciencia", ["Académico", "Divulgación", "Fauna"])

biblio.agregar_seccion("FILO", "Filosofía")

biblio.agregar_seccion("FISI", "Física", ["Astronomía", "General", "Teórica"])

biblio.agregar_seccion("GEOG", "Geografía", ["Viajes", "Mapas"])

biblio.agregar_seccion("HIST", "Historia")

biblio.agregar_seccion("IDIO", "Idiomas")

biblio.agregar_seccion("JUVE", "Juvenil", ["Cómics", "Conocimiento", "Novela"])
biblio.agregar_seccion("MITO", "Mitología")

biblio.agregar_seccion("NOVE", "Novela", ["Histórica", "Suspense", "Romántica", "Aventura", "Bélica", "Ciencia Ficción", "Ficción Social", "Realismo Social", "Fantástica"]) 

biblio.agregar_seccion("OPOS", "Oposiciones")

biblio.agregar_seccion("REFE", "Referencias")

biblio.agregar_seccion("RELI", "Religión")

biblio.agregar_seccion("ZZZZ", "Apuntes, notas y varios")

In [21]:
# Agregar libros

# ==== AJEDREZ ====
biblio.agregar_libro("AJED", "El arte del estudio de Ajedrez", "Zoilo R. Caputto")
biblio.agregar_libro("AJED", "El gran libro de ajedrez", "Natale Ramini")
biblio.agregar_libro("AJED", "Larousse del Ajedrez", "Larousse")
biblio.agregar_libro("AJED", "Ajedrez Español", "Editorial SEAS")
biblio.agregar_libro("AJED", "Las batallas de las ideas en el Ajedrez", "Anthony Saidy")
biblio.agregar_libro("AJED", "Ajedrez, maestro contra Amateur", "M. Euwe y W. Meiden")
biblio.agregar_libro("AJED", "El arte del Sacrificio en el ajedrez", "Editorial de Vecchi")
biblio.agregar_libro("AJED", "The Ultimate Pirc", "John Nunn")
biblio.agregar_libro("AJED", "Premios de belleza en el Ajedrez", "François Le Lionnais")
biblio.agregar_libro("AJED", "Táctica moderna en el Ajedrez", "Ludek Pachmann")
biblio.agregar_libro("AJED", "Aperturas Semiabiertas I", "Ludek Pachmann")
biblio.agregar_libro("AJED", "Aperturas Abiertas", "Ludek Pachmann")
biblio.agregar_libro("AJED", "Táctica moderna en el Ajedrez (repetido)", "Ludek Pachmann")
biblio.agregar_libro("AJED", "Aperturas Semiabiertas II", "Ludek Pachmann")
biblio.agregar_libro("AJED", "Garry Kasparov", "M. Yudovich")
biblio.agregar_libro("AJED", "El Ajedrez", "Ricardo Aguilera")
biblio.agregar_libro("AJED", "Corona Mundial de Ajedrez Kasparov-Karpov", "Inaplicable")
biblio.agregar_libro("AJED", "Los principios del Ajedrez", "Ray Arvid")
biblio.agregar_libro("AJED", "La partida de Ajedrez", "Rubinstein")
biblio.agregar_libro("AJED", "Defensa Siciliana", "G. Kasparov y N. Nikitin (Club del Ajedrez)")
biblio.agregar_libro("AJED", "El estilo posicional", "V. Smyslov")
biblio.agregar_libro("AJED", "El error en la apertura", "Club del Ajedrez")
biblio.agregar_libro("AJED", "Decisiones fundamentales del Ajedrez Capablanca", "Club del Ajedrez")
biblio.agregar_libro("AJED", "Fundamentos del Ajedrez Capablanca", "Club del Ajedrez")
biblio.agregar_libro("AJED", "Aperturas Cerradas", "Ludek Pachmann")
biblio.agregar_libro("AJED", "Cuarto libro de Ajedrez", "Fred Reinfeld")
biblio.agregar_libro("AJED", "Apertura Italiana, 250 partidas", "Colección Enroque")
biblio.agregar_libro("AJED", "Mitología del ajedrez", "Francesc Lluis Cardona")

# ==== ARTE / CINE ====
biblio.agregar_libro("ARTE", "Historia del cine", "Roman Gubern", "Cine")
biblio.agregar_libro("ARTE", "Días de una cámara", "Néstor Almendros", "Cine")
biblio.agregar_libro("ARTE", "Plano a plano, de la idea a la pantalla", "S. D. Katz", "Cine")
biblio.agregar_libro("ARTE", "Técnica del montaje cinematográfico", "Karl Reitz", "Cine")
biblio.agregar_libro("ARTE", "Cinema como arma de clase", "Carlos y David Peez Merinero", "Cine")
biblio.agregar_libro("ARTE", "El arte de actuar", "UTAHAGEN", "Cine")
biblio.agregar_libro("ARTE", "Relatos que me asustaron", "Alfred Hitchcock", "Cine")
biblio.agregar_libro("ARTE", "Conversaciones íntimas", "Ingmar Bergman", "Cine")
biblio.agregar_libro("ARTE", "Boda en el delta", "Eudora Welty Proulx", "Cine")
biblio.agregar_libro("ARTE", "De aquí a la eternidad", "James Jones", "Cine")
biblio.agregar_libro("ARTE", "Marilyn Monroe", "Selmay ediciones", "Cine")
biblio.agregar_libro("ARTE", "El libro de Orson Welles", "El País", "Cine")
biblio.agregar_libro("ARTE", "Vinidiana", "Luis Buñuel", "Cine")
biblio.agregar_libro("ARTE", "Los colosos del cine, Paul Newman", "Cinema club colection", "Cine")
biblio.agregar_libro("ARTE", "Los colosos del cine, Alfred Hitchcock", "Cinema club colection", "Cine")
biblio.agregar_libro("ARTE", "Los colosos del cine, Gary Grant", "Cinema club colection", "Cine")
biblio.agregar_libro("ARTE", "Los colosos del cine, J.L. Mankiewrcz", "Cinema club colection", "Cine")
biblio.agregar_libro("ARTE", "Los colosos del cine, Roman Polansky", "Cinema club colection", "Cine")
biblio.agregar_libro("ARTE", "Los colosos del cine, Marlon Brando", "Cinema club colection", "Cine")
biblio.agregar_libro("ARTE", "Brando por Brando", "Fotogramas", "Cine")
biblio.agregar_libro("ARTE", "Sin plumas", "Woody Allen", "Cine")
biblio.agregar_libro("ARTE", "Casablanca", "El País", "Cine")
biblio.agregar_libro("ARTE", "Alfred Hitchcock. Psicosis", "Edición de oro", "Cine")
biblio.agregar_libro("ARTE", "Como acabar de una vez por todas con la cultura", "Woody Allen", "Cine")
biblio.agregar_libro("ARTE", "The end", "Jean Claude Carrière", "Cine")
biblio.agregar_libro("ARTE", "Montgomery Clif", "Patricia Bosworth", "Cine")
biblio.agregar_libro("ARTE", "Cine y literatura", "Agenda cultural 1999", "Cine")
biblio.agregar_libro("ARTE", "Folleto escuela de Artes Visuales", "No aplicable", "Cine")
biblio.agregar_libro("ARTE", "Fiesta, Ernest Hemingwaye", "Best Sellers Planeta", "Cine")


# ==== ARTE / MUSEOS ====
biblio.agregar_libro("ARTE", "La guía del Prado", "Museo Nacional del Prado", "Museos")
biblio.agregar_libro("ARTE", "La luz del Nácar", "Museo Avenida de América", "Museos")
biblio.agregar_libro("ARTE", "Guía British Museum", "Museo Británico de Londres", "Museos")
biblio.agregar_libro("ARTE", "Real Monasterio de San Lorenzo del Escorial", "José Luis Sancho Gaspar", "Museos")
biblio.agregar_libro("ARTE", "Museo del Romanticismo - La colección", "Museo del Romanticismo", "Museos")
biblio.agregar_libro("ARTE", "La Alhambra - Arquitectura, historia, planos y leyenda", "Ana Sánchez y Ángel Sánchez", "Museos")
biblio.agregar_libro("ARTE", "Guía The National Gallery", "Erika Langmuir", "Museos")
biblio.agregar_libro("ARTE", "Guía Los vikingos", "Museo de Oslo", "Museos")
biblio.agregar_libro("ARTE", "Museo de arte ruso. Pinturas", "No aplicable", "Museos")
biblio.agregar_libro("ARTE", "Museo Sorolla", "No aplicable", "Museos")
biblio.agregar_libro("ARTE", "Cuadernos del M.A.N. Grecia", "No aplicable", "Museos")
biblio.agregar_libro("ARTE", "La Acrópolis. El museo de la Acrópolis", "Ekdotike Athenon", "Museos")
biblio.agregar_libro("ARTE", "Entender mejor la pintura en la museo de Orsay", "No aplicable", "Museos")
biblio.agregar_libro("ARTE", "En el mar de Sorolla con Manuel Vicent", "No aplicable", "Museos")
biblio.agregar_libro("ARTE", "Guía, Catedral de Siena", "No aplicable", "Museos")
biblio.agregar_libro("ARTE", "Guía, Ciudad de Vaticano", "Edizione Musei Vaticani", "Museos")
biblio.agregar_libro("ARTE", "Guía, La civilización Minoica", "Sosso Logiabov Platonos", "Museos")

# ==== ARTE / MÚSICA ====
biblio.agregar_libro("ARTE", "Arias de concierto, Mozart (CD)", "El País", "Música")
biblio.agregar_libro("ARTE", "Colección genios de la música: Bach (CD)", "No aplicable", "Música")
biblio.agregar_libro("ARTE", "Colección genios de la música: Beethoven (CD)", "No aplicable", "Música")
biblio.agregar_libro("ARTE", "Colección genios de la música: Tchaikovsky (CD)", "No aplicable", "Música")
biblio.agregar_libro("ARTE", "Colección genios de la música: Wagner (CD)", "No aplicable", "Música")
biblio.agregar_libro("ARTE", "Colección genios de la música: Verdi (CD)", "No aplicable", "Música")
biblio.agregar_libro("ARTE", "Colección genios de la música: Haendel (CD)", "No aplicable", "Música")
biblio.agregar_libro("ARTE", "Colección genios de la música: Haydn (CD)", "No aplicable", "Música")
biblio.agregar_libro("ARTE", "Colección genios de la música: Mozart (CD)", "No aplicable", "Música")
biblio.agregar_libro("ARTE", "Colección genios de la música: Strauss (CD)", "No aplicable", "Música")
biblio.agregar_libro("ARTE", "Las cuatro estaciones, Vivaldi (CD)", "Ed. Deutsche Grammophon", "Música")
biblio.agregar_libro("ARTE", "Gran Mozart, los años dorados", "H.C. Robbins Landon", "Música")
biblio.agregar_libro("ARTE", "Cancionero musical y poético: tonos castellanos", "Gerardo Arriaga", "Música")
biblio.agregar_libro("ARTE", "Modos vetus", "Ed. Edelvives", "Música")
biblio.agregar_libro("ARTE", "Choralgesänge", "J.S. Bach", "Música")
biblio.agregar_libro("ARTE", "Lézioni di Paleografía", "Claudio Battelli", "Música")
biblio.agregar_libro("ARTE", "Antón García Abril: Sonidos de libertad", "J. Cabezón Marín", "Música")
biblio.agregar_libro("ARTE", "Zarzuelas y teatro", "Ricardo de la Vega", "Música")
biblio.agregar_libro("ARTE", "Pequeña antología coral", "Marcos Vega", "Música")
biblio.agregar_libro("ARTE", "Apreciación musical", "Pedro Machado de Castro", "Música")
biblio.agregar_libro("ARTE", "El caso Vivaldi", "Federico María Sardelli", "Música")
biblio.agregar_libro("ARTE", "Graduale Triplex", "No aplicable", "Música")
biblio.agregar_libro("ARTE", "Historia de la música española", "Ed. Alianza Música", "Música")
biblio.agregar_libro("ARTE", "El ruido eterno", "Alex Ross", "Música")
biblio.agregar_libro("ARTE", "Música clásica", "Ruthston", "Música")
biblio.agregar_libro("ARTE", "Relatos célebres sobre la música", "Varios autores", "Música")
biblio.agregar_libro("ARTE", "La música andalusí", "Amin Chaachoo", "Música")
biblio.agregar_libro("ARTE", "Diccionario enciclopédico de la música, 1.A", "Editorial Rambos", "Música")
biblio.agregar_libro("ARTE", "Historia de la música", "Sopeña", "Música")
biblio.agregar_libro("ARTE", "La ópera y Sevilla", "No aplicable", "Música")
biblio.agregar_libro("ARTE", "Western Pianchant", "Hiley", "Música")
biblio.agregar_libro("ARTE", "Historia de la música occidental", "Grout, Palisca", "Música")
biblio.agregar_libro("ARTE", "El canto mozárabe y su entorno", "Varios autores", "Música")

# ==== ARTE / PINTURA ====
biblio.agregar_libro("ARTE", "La pintura Italiana I", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "La pintura Italiana II", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "La pintura Italiana III", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "La pintura Española I", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "La pintura Española II", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "La pintura Española III", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "Grandes temas de la pintura: El retrato", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "Grandes temas de la pintura: El paisaje", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "Grandes temas de la pintura: El bodegón", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "Grandes temas de la pintura: La pintura de interiores", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "Grandes temas de la pintura: La pintura religiosa", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "Los genios universales de la pintura: Monet", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "Los genios universales de la pintura: Matisse", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "Los genios universales de la pintura: Caravaggio", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "Grandes maestros de la pintura: Picasso", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "El Impresionismo: Van Gogh", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "Goya", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "Maestros de la pintura mundial: Alfred Sisley", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "Maestros de la pintura mundial: Bartolomé Esteban Murillo", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "Cézanne", "No Aplicable", "Pintura")
biblio.agregar_libro("ARTE", "Cien obras maestras de la pintura", "Marcial Olivar", "Pintura")

# ==== ARTE / POESÍA ====
biblio.agregar_libro("ARTE", "Obras Completas", "Gustavo Adolfo Bécquer", "Poesía")
biblio.agregar_libro("ARTE", "Biblioteca de Poesía", "Rafael Alberti", "Poesía")
biblio.agregar_libro("ARTE", "Poesía completa (1960-1997)", "Francisco Brines", "Poesía")
biblio.agregar_libro("ARTE", "Libro de Poemas (1921)", "Federico García Lorca", "Poesía")
biblio.agregar_libro("ARTE", "Poesía Completa Volumen I", "Luis Cernuda", "Poesía")
biblio.agregar_libro("ARTE", "Antología de poesía austriaca actual", "Función Lenguaje", "Poesía")
biblio.agregar_libro("ARTE", "Antología de poesía suiza actual", "Fuerza Language", "Poesía")
biblio.agregar_libro("ARTE", "Eugenio Oneguin", "Alexander S. Pushkin", "Poesía")
biblio.agregar_libro("ARTE", "Poesías completas", "Antonio Machado", "Poesía")
biblio.agregar_libro("ARTE", "Antología Poética", "Antonio Machado", "Poesía")
biblio.agregar_libro("ARTE", "Poesía", "Antonio Machado", "Poesía")
biblio.agregar_libro("ARTE", "Poemas 1913-16", "Maya Kowsky", "Poesía")
biblio.agregar_libro("ARTE", "Nueva Antología Poética", "Rainer Maria Rilke", "Poesía")
biblio.agregar_libro("ARTE", "Bodas de sangre", "Federico García Lorca", "Poesía")
biblio.agregar_libro("ARTE", "Hijos de la Ira", "Dámaso Alonso", "Poesía")
biblio.agregar_libro("ARTE", "Antología", "Gustavo Adolfo Bécquer", "Poesía")
biblio.agregar_libro("ARTE", "Poesía, palabra en el tiempo", "Antonio Machado", "Poesía")
biblio.agregar_libro("ARTE", "El hombre y su poesía", "Miguel Hernández", "Poesía")
biblio.agregar_libro("ARTE", "Veinte poemas de amor y una canción desesperada", "Pablo Neruda", "Poesía")
biblio.agregar_libro("ARTE", "Antología poética", "Juan Ramón Jiménez", "Poesía")
biblio.agregar_libro("ARTE", "De todo corazón 111 poemas de amor", "Editorial S.L.", "Poesía")
biblio.agregar_libro("ARTE", "Postales de escritores y poetas rusos", "Postales", "Poesía")
biblio.agregar_libro("ARTE", "Antología poética", "Antonio Machado", "Poesía")


# ==== ARTE / TEATRO ====
biblio.agregar_libro("ARTE", "La Celestina", "Fernando de Rojas", "Teatro")
biblio.agregar_libro("ARTE", "La vida es sueño", "Pedro Calderón de la Barca", "Teatro")
biblio.agregar_libro("ARTE", "La venganza del Don Mendo", "Muñoz Seca", "Teatro")
biblio.agregar_libro("ARTE", "El perro del hortelano", "Lope de Vega", "Teatro")
biblio.agregar_libro("ARTE", "Fausto", "Goethe", "Teatro")
biblio.agregar_libro("ARTE", "El caballero de Olmedo", "Lope de Vega", "Teatro")
biblio.agregar_libro("ARTE", "El Barbero de Sevilla", "Beaumarchais", "Teatro")
biblio.agregar_libro("ARTE", "Historia de una escalera", "Antonio Buero Vallejo", "Teatro")
biblio.agregar_libro("ARTE", "El alcalde de Zalamea", "Calderón de la Barca - Editorial Ramón Sopena", "Teatro")
biblio.agregar_libro("ARTE", "Casa de muñecas / El pato salvaje", "Henrik Ibsen", "Teatro")
biblio.agregar_libro("ARTE", "El Avaro / El enfermo imaginario", "Molière", "Teatro")
biblio.agregar_libro("ARTE", "Romeo y Julieta / Júlio César", "William Shakespeare", "Teatro")
biblio.agregar_libro("ARTE", "El enfermo imaginario / El médico a palos", "Molière", "Teatro")
biblio.agregar_libro("ARTE", "Los intereses creados", "Jacinto Benavente", "Teatro")
biblio.agregar_libro("ARTE", "Fuenteovejuna / El perro sin vergüenza", "Lope de Vega", "Teatro")
biblio.agregar_libro("ARTE", "Teatro universal escogido", "Editorial El Corte Inglés", "Teatro")
biblio.agregar_libro("ARTE", "Don Juan Tenorio", "Zorrilla", "Teatro")
biblio.agregar_libro("ARTE", "Las bodas de Fígaro", "Beaumarchais", "Teatro")
biblio.agregar_libro("ARTE", "Historia básica del arte escénico", "César Oliva", "Teatro")
biblio.agregar_libro("ARTE", "La zapatera prodigiosa", "Federico García Lorca", "Teatro")
biblio.agregar_libro("ARTE", "Hamlet", "William Shakespeare - Ed. Salvat", "Teatro")
biblio.agregar_libro("ARTE", "Cancionero y Romancero español", "Francisco Alonso", "Teatro")
biblio.agregar_libro("ARTE", "Entremeses", "Miguel de Cervantes", "Teatro")
biblio.agregar_libro("ARTE", "Hamlet / Macbeth", "Shakespeare", "Teatro")
biblio.agregar_libro("ARTE", "Hamlet", "Shakespeare (Ed. Cátedra Letras Universales)", "Teatro")
biblio.agregar_libro("ARTE", "Casandra", "Benito Pérez Galdós", "Teatro")
biblio.agregar_libro("ARTE", "Don Álvaro o la fuerza del sino", "Duque de Rivas", "Teatro")
biblio.agregar_libro("ARTE", "Tartufo, Don Juan", "Moliere", "Teatro")
biblio.agregar_libro("ARTE", "La dama del alba", "Alejandro Casona", "Teatro")
biblio.agregar_libro("ARTE", "Retorno al hogar", "Lajos Zilahy", "Teatro")
biblio.agregar_libro("ARTE", "El sueño de una noche de verano, Las alegres comadres de Windsor", "Shakespeare", "Teatro")
biblio.agregar_libro("ARTE", "María Pineda, Doña Rosita la soltera o el lenguaje de las flores", "Federico García Lorca", "Teatro")
biblio.agregar_libro("ARTE", "El alcalde de Zalamea", "Calderón de la Barca - Editorial Edimat libros", "Teatro")

# ==== BIENESTAR ====
biblio.agregar_libro("BIEN", "Grandes misterios de las sociedades secretas", "Ramiro A. Calle")
biblio.agregar_libro("BIEN", "En busca del Faquir", "Ramiro A. Calle")
biblio.agregar_libro("BIEN", "Ser feliz en Alaska", "Rafael Santandreu")
biblio.agregar_libro("BIEN", "Alegría, la felicidad que surge del interior", "Editorial Grijalbo")
biblio.agregar_libro("BIEN", "1080 Recetas de Cocina", "Simone Ortega")
biblio.agregar_libro("BIEN", "Pilates", "Mais Wesson y Mark Lauren")
biblio.agregar_libro("BIEN", "Estirándose, el libro de fitness más famoso de todos los tiempos", "Bob Anderson")
biblio.agregar_libro("BIEN", "El mito de la diosa fortuna", "Jorge Bucay")
biblio.agregar_libro("BIEN", "Seso quebrado", "Ramiro A. Calle")
biblio.agregar_libro("BIEN", "Constelaciones", "Guía ilustrada de astrología")
biblio.agregar_libro("BIEN", "Construye tu destino", "Wayne W. Dyer")
biblio.agregar_libro("BIEN", "La voz de tu alma", "Lain")
biblio.agregar_libro("BIEN", "Diccionario práctico de plantas y jardines", "Ediciones Nobel")
biblio.agregar_libro("BIEN", "El Mundo Amarillo", "Albert Espinosa")
biblio.agregar_libro("BIEN", "El Don de la felicidad", "Elaine N. Aron")
biblio.agregar_libro("BIEN", "Tus zonas erróneas", "Dr. Wayne W. Dyer")
biblio.agregar_libro("BIEN", "Inteligencia Emocional", "Daniel Goleman")
biblio.agregar_libro("BIEN", "El Tercer ojo", "T. Lobsang Rampa")
biblio.agregar_libro("BIEN", "El buen amor", "Sergio Sinay")
biblio.agregar_libro("BIEN", "El arte de vivir en pareja", "Sergio Sinay")
biblio.agregar_libro("BIEN", "Yoga teoría y práctica", "Francesca Chiappone")
biblio.agregar_libro("BIEN", "Mística oriental para occidentales", "Panico A. Calle")
biblio.agregar_libro("BIEN", "Aprendices de las emociones", "Norberto Levy")
biblio.agregar_libro("BIEN", "El amor que nos cura", "Boris Cyrulnik")
biblio.agregar_libro("BIEN", "Manual práctico de chacras", "Mika Widmaaka")
biblio.agregar_libro("BIEN", "El sinsentido común", "Borja Vilaseca")
biblio.agregar_libro("BIEN", "La cocina saludable", "Moritz Anton")
biblio.agregar_libro("BIEN", "Aromaterapia", "D. Rose Fiszbein")
biblio.agregar_libro("BIEN", "Mindfulness", "Mark Williams")
biblio.agregar_libro("BIEN", "El huerto y la botica en casa", "Carmel Conche")
biblio.agregar_libro("BIEN", "Medicina natural para niños", "Julian Scotti")
biblio.agregar_libro("BIEN", "Cómo practicar la meditación", "Centro Sivamanda")
biblio.agregar_libro("BIEN", "La alimentación de tu hijo", "Patricia Castillo")
biblio.agregar_libro("BIEN", "El libro del Yogurt", "Susaeta")
biblio.agregar_libro("BIEN", "Manual de plantas de interior", "Everest")
biblio.agregar_libro("BIEN", "Gran libro del masaje", "Servilibro")
biblio.agregar_libro("BIEN", "El Tao", "Javid Ru")
biblio.agregar_libro("BIEN", "El Mundo en tus manos", "Elsa Punset")
biblio.agregar_libro("BIEN", "Felicidades", "Elsa Punset")
biblio.agregar_libro("BIEN", "El viaje a la Felicidad", "Eduardo Punset")
biblio.agregar_libro("BIEN", "Juego para ejercitar la mente", "Kit integral")
biblio.agregar_libro("BIEN", "La dieta ayurvédica", "Editorial Hispano Europea")
biblio.agregar_libro("BIEN", "Guía de la meditación", "E. Ko Editable")
biblio.agregar_libro("BIEN", "Recetas vegetarianas del mundo", "Sanch Brown")
biblio.agregar_libro("BIEN", "Manual de los vinos de España", "Pedro Plasencia")
biblio.agregar_libro("BIEN", "Cocinar con verduras", "Cocina actual del siglo XXI")
biblio.agregar_libro("BIEN", "La Cocina vegetariana clásica", "Roue Eliot")
biblio.agregar_libro("BIEN", "Cocina para los más pequeños", "España Calpe")
biblio.agregar_libro("BIEN", "La digestión es la cuestión", "Giulia Enders")
biblio.agregar_libro("BIEN", "El alma está en el cerebro", "Eduardo Punset")
biblio.agregar_libro("BIEN", "Si tú me dices ven lo dejo todo... pero dime ven", "Albert Espinosa")
biblio.agregar_libro("BIEN", "El arte de no amargarse la vida", "Rafael Santandreu")
biblio.agregar_libro("BIEN", "100 años de nutrición", "Claudio Mariscal")
biblio.agregar_libro("BIEN", "El vino no tiene misterio", "Ophelie Neiman")
biblio.agregar_libro("BIEN", "La Miel", "Janet Bond")
biblio.agregar_libro("BIEN", "La manzanilla", "Valter Curzi")
biblio.agregar_libro("BIEN", "Manual para el cuidado del perro", "Nuseta")
biblio.agregar_libro("BIEN", "Un jardín muy divertido", "Erik Luguet")
biblio.agregar_libro("BIEN", "El arte de cuidarte a las 5 estaciones", "Blanca Gadea")
biblio.agregar_libro("BIEN", "Ejercicios de yoga", "S. Muzdabar")
biblio.agregar_libro("BIEN", "Tantra, energía y éxtasis", "Osho")
biblio.agregar_libro("BIEN", "El gran libro de la madre", "No aplicable")
biblio.agregar_libro("BIEN", "El llamado de los árboles", "Antonio Gomez")
biblio.agregar_libro("BIEN", "La alegría de vivir", "Crisom Swett Manson")
biblio.agregar_libro("BIEN", "Mi cerebro y yo", "José M.R. Delgado")
biblio.agregar_libro("BIEN", "La vida... ¿cómo se presta aquí?", "No aplicable")
biblio.agregar_libro("BIEN", "El libro del amor", "Editorial Planeta")
biblio.agregar_libro("BIEN", "En busca del Kamasutra", "Editorial Planeta")
biblio.agregar_libro("BIEN", "La botica de la abuela", "Editorial Planeta")

# ==== CIENCIA / ACADÉMICO ====
biblio.agregar_libro("CIEN", "Física 2º Bachillerato", "McGraw Hill", "Académico")
biblio.agregar_libro("CIEN", "Física Tipler-Mosca Volumen 1A Mecánica", "Tipler - Mosca", "Académico")
biblio.agregar_libro("CIEN", "Física Tipler-Mosca Volumen 1B Oscilaciones y ondas", "Tipler - Mosca", "Académico")
biblio.agregar_libro("CIEN", "Física Tipler-Mosca Volumen 1C Termodinámica", "Tipler - Mosca", "Académico")
biblio.agregar_libro("CIEN", "Física Tipler-Mosca Volumen 2A Electromagnetismo", "Tipler - Mosca", "Académico")
biblio.agregar_libro("CIEN", "Física Tipler-Mosca Volumen 2B Luz", "Tipler - Mosca", "Académico")
biblio.agregar_libro("CIEN", "Física Tipler-Mosca Física moderna", "Tipler - Mosca", "Académico")
biblio.agregar_libro("CIEN", "Física Universitaria Volumen 1", "Zemansky", "Académico")
biblio.agregar_libro("CIEN", "Física Universitaria Volumen 2", "Zemansky", "Académico")
biblio.agregar_libro("CIEN", "Matemáticas Básicas", "UNED", "Académico")
biblio.agregar_libro("CIEN", "Cálculo Vectorial", "Marsden - Tromba", "Académico")
biblio.agregar_libro("CIEN", "Álgebra Lineal", "Juan de Burgos", "Académico")
biblio.agregar_libro("CIEN", "Manual de Matemáticas para las Ciencias Médicas", "Psipkin", "Académico")
biblio.agregar_libro("CIEN", "Geometría Moderna. Método de la Teoría de Homologías", "Dubrovín", "Académico")
biblio.agregar_libro("CIEN", "Álgebra Práctica", "Carlos Matías Aracil", "Académico")
biblio.agregar_libro("CIEN", "Electrotecnia para Técnicos", "J. A. Martín Terneiro", "Académico")
biblio.agregar_libro("CIEN", "Principia Mathematica", "Isaac Newton", "Académico")
biblio.agregar_libro("CIEN", "Física General", "Lozano", "Académico")
biblio.agregar_libro("CIEN", "Tratado Elemental de Física", "Ganot", "Académico")
biblio.agregar_libro("CIEN", "Introducción a la Física de Átomos y Moléculas", "Francisco Blanco Ramos", "Académico")
biblio.agregar_libro("CIEN", "Curso de Astronomía General", "Bakulin", "Académico")
biblio.agregar_libro("CIEN", "Manual de Física General", "B. I. Yavorski", "Académico")
biblio.agregar_libro("CIEN", "Curso de Física General", "S. Faish", "Académico")
biblio.agregar_libro("CIEN", "Álgebra Lineal y algunas de sus aplicaciones", "N. I. Golovina", "Académico")
biblio.agregar_libro("CIEN", "Ciencia Moderna y Filosofía", "José María Riaga", "Académico")
biblio.agregar_libro("CIEN", "Problemas de Física", "Aguilar", "Académico")
biblio.agregar_libro("CIEN", "Problemas de Física 1", "Gullón", "Académico")
biblio.agregar_libro("CIEN", "Problemas de Física 3", "Gullón", "Académico")
biblio.agregar_libro("CIEN", "Problemas de Física", "Precioso", "Académico")
biblio.agregar_libro("CIEN", "Problemas seleccionados de Física Elemental", "B. B. Biyatkeev", "Académico")
biblio.agregar_libro("CIEN", "Compendio de Física y Química", "Aleicha", "Académico")
biblio.agregar_libro("CIEN", "Química General", "B. V. Ketkánov", "Académico")
biblio.agregar_libro("CIEN", "Temas básicos de Química", "S. Donzillo", "Académico")
biblio.agregar_libro("CIEN", "Química Orgánica para Enseñanza Media", "L. A. Tsvetkov", "Académico")
biblio.agregar_libro("CIEN", "Bioquímica", "Lehninger", "Académico")
biblio.agregar_libro("CIEN", "Prácticas de Química General Inorgánica", "V. Semosthin", "Académico")
biblio.agregar_libro("CIEN", "Curso de Mineralogía", "A. Betekhtin", "Académico")
biblio.agregar_libro("CIEN", "Tratado de Terapéutica y Materia Médica 1872", "A. Trousseau, H. Pidoux", "Académico")
biblio.agregar_libro("CIEN", "Cálculo diferencial", "Víctor Sánchez", "Académico")
biblio.agregar_libro("CIEN", "Tratado de geometría, tercer grado", "No aplicable", "Académico")
biblio.agregar_libro("CIEN", "Nociones de álgebra", "Bruño", "Académico")
biblio.agregar_libro("CIEN", "Elemnetos de matemáticas", "Borges", "Académico")

# ==== CIENCIA / DIVULGACIÓN ====
biblio.agregar_libro("CIEN", "Destino año 2000", "Pierre Pailhé", "Divulgación")
biblio.agregar_libro("CIEN", "Cómo fueron descubiertos los elementos químicos", "D.N. Trífonov", "Divulgación")
biblio.agregar_libro("CIEN", "Yo, Simpson y las matemáticas", "Simon Singh", "Divulgación")
biblio.agregar_libro("CIEN", "Matemática para divertirse", "Martin Gardner", "Divulgación")
biblio.agregar_libro("CIEN", "Un eterno y grácil bucle: Gödel, Escher, Bach", "Douglas Hofstadter", "Divulgación")
biblio.agregar_libro("CIEN", "Colección revistas: Tribunal de Astronomía (31 ejemplares)", "Tribunal de Astronomía", "Divulgación")
biblio.agregar_libro("CIEN", "Colección revistas: Investigación y Ciencia (36 ejemplares)", "Investigación y Ciencia", "Divulgación")
biblio.agregar_libro("CIEN", "Colección revistas: Revista española de física (10 ejemplares)", "Revista española de física", "Divulgación")
biblio.agregar_libro("CIEN", "Colección revistas: The Messenger (4 ejemplares)", "The Messenger", "Divulgación")
biblio.agregar_libro("CIEN", "Colección revistas: National Geographic (5 ejemplares)", "National Geographic", "Divulgación")
biblio.agregar_libro("CIEN", "La enciclopedia del estudiante. Ciencia, la Tierra y el Universo", "El País", "Divulgación")
biblio.agregar_libro("CIEN", "Libro de la ciencia", "Eduardo Vilas", "Divulgación")
biblio.agregar_libro("CIEN", "Lecciones populares de matemáticas: Construcciones geométricas mediante un compás", "A.N. Kostovski", "Divulgación")
biblio.agregar_libro("CIEN", "Lecciones populares de matemáticas: Acerca de la demostración en geometría", "A.I. Fetísov", "Divulgación")
biblio.agregar_libro("CIEN", "Lecciones populares de matemáticas: ¿Qué es el cálculo diferencial?", "V.G. Boltianski", "Divulgación")
biblio.agregar_libro("CIEN", "Lecciones populares de matemáticas: Análisis matemático en el campo de funciones racionales", "G.E. Shílov", "Divulgación")
biblio.agregar_libro("CIEN", "Lecciones populares de matemáticas: Inducción en la geometría", "L.I. Goloviná", "Divulgación")
biblio.agregar_libro("CIEN", "Lecciones populares de matemáticas: Construcciones geométricas mediante un compás", "A.N. Kostovski", "Divulgación")

# ==== CIENCIA / FAUNA ====
biblio.agregar_libro("CIEN", "Fauna Ibérica, El hombre y la Tierra I", "Félix Rodríguez de la Fuente", "Fauna")
biblio.agregar_libro("CIEN", "Fauna Ibérica, El hombre y la Tierra II", "Félix Rodríguez de la Fuente", "Fauna")
biblio.agregar_libro("CIEN", "Fauna Ibérica, El hombre y la Tierra III", "Félix Rodríguez de la Fuente", "Fauna")
biblio.agregar_libro("CIEN", "Fauna Ibérica, El hombre y la Tierra IV", "Félix Rodríguez de la Fuente", "Fauna")
biblio.agregar_libro("CIEN", "Fauna Ibérica, El hombre y la Tierra V", "Félix Rodríguez de la Fuente", "Fauna")
biblio.agregar_libro("CIEN", "Fauna Ibérica, El hombre y la Tierra VI", "Félix Rodríguez de la Fuente", "Fauna")
biblio.agregar_libro("CIEN", "Fauna Ibérica, El hombre y la Tierra VII", "Félix Rodríguez de la Fuente", "Fauna")
biblio.agregar_libro("CIEN", "Fauna Ibérica, El hombre y la Tierra VIII", "Félix Rodríguez de la Fuente", "Fauna")
biblio.agregar_libro("CIEN", "Fauna Ibérica, El hombre y la Tierra IX", "Félix Rodríguez de la Fuente", "Fauna")
biblio.agregar_libro("CIEN", "Fauna Ibérica, El hombre y la Tierra X", "Félix Rodríguez de la Fuente", "Fauna")

# ==== FILOSOFÍA ====
biblio.agregar_libro("FILO", "Los filósofos presocráticos", "G.S. Kirk y J.E. Raven")
biblio.agregar_libro("FILO", "Sobre el ordenamiento de la naturaleza", "Parménides")
biblio.agregar_libro("FILO", "Epicuro", "Antonio Roldán Marco")
biblio.agregar_libro("FILO", "Pitágoras esencial", "Abraham Miguel Ponz")
biblio.agregar_libro("FILO", "Apología de Sócrates", "Platón")
biblio.agregar_libro("FILO", "Política", "Aristóteles")
biblio.agregar_libro("FILO", "Categorías. De interpretatione / Isagoge", "Aristóteles / Porfirio")
biblio.agregar_libro("FILO", "Ética a Nicómaco", "Aristóteles")
biblio.agregar_libro("FILO", "La lógica del pensamiento. Aristóteles", "Antonio Roldán Marco")
biblio.agregar_libro("FILO", "Poética", "Aristóteles")
biblio.agregar_libro("FILO", "Retórica", "Aristóteles")
biblio.agregar_libro("FILO", "Parménides", "Platón")
biblio.agregar_libro("FILO", "La República", "Platón")
biblio.agregar_libro("FILO", "Epicteto. Un manual para la vida", "Antonio Roldán Marco")
biblio.agregar_libro("FILO", "Vidas de filósofos ilustres", "Diógenes Laercio")
biblio.agregar_libro("FILO", "Hipatia", "Clelia Martigni Maza")
biblio.agregar_libro("FILO", "Meditaciones", "Marco Aurelio")
biblio.agregar_libro("FILO", "Séneca o el beneficio de la duda", "Antonio Gala")
biblio.agregar_libro("FILO", "Meditaciones metafísicas", "René Descartes")
biblio.agregar_libro("FILO", "Discurso de metafísica", "Leibniz")
biblio.agregar_libro("FILO", "Novum Organum", "Francis Bacon")
biblio.agregar_libro("FILO", "El sueño del humanismo. De Petrarca a Erasmo", "Francisco Rico")
biblio.agregar_libro("FILO", "Pensar y no caer", "Ramón Andrés")
biblio.agregar_libro("FILO", "El sofista", "Platón")
biblio.agregar_libro("FILO", "Sobre la felicidad", "Séneca")
biblio.agregar_libro("FILO", "Kant, vida y doctrina", "Ernst Cassirer")
biblio.agregar_libro("FILO", "Fundamentación de la metafísica de las costumbres", "Immanuel Kant")
biblio.agregar_libro("FILO", "El contrato social", "Jean-Jacques Rousseau")
biblio.agregar_libro("FILO", "Investigación sobre el conocimiento humano", "David Hume")
biblio.agregar_libro("FILO", "Introducción a Newton", "Mauricio Mamiani")
biblio.agregar_libro("FILO", "Lecciones sobre la filosofía de la historia universal", "G.W.F. Hegel")
biblio.agregar_libro("FILO", "Fenomenología del espíritu", "G.W.F. Hegel")
biblio.agregar_libro("FILO", "Humano, demasiado humano", "Friedrich Nietzsche")
biblio.agregar_libro("FILO", "Así habló Zaratustra", "Friedrich Nietzsche")
biblio.agregar_libro("FILO", "El nacimiento de la tragedia", "Friedrich Nietzsche")
biblio.agregar_libro("FILO", "La genealogía de la moral", "Friedrich Nietzsche")
biblio.agregar_libro("FILO", "El ocaso de los ídolos", "Friedrich Nietzsche")
biblio.agregar_libro("FILO", "Schopenhauer, Nietzsche, Freud", "Thomas Mann")
biblio.agregar_libro("FILO", "El Capital. Libro I. Tomo I", "Karl Marx")
biblio.agregar_libro("FILO", "El Capital. Libro I. Tomo II", "Karl Marx")
biblio.agregar_libro("FILO", "El Capital. Libro I. Tomo III", "Karl Marx")
biblio.agregar_libro("FILO", "El Capital. Libro II. Tomo I", "Karl Marx")
biblio.agregar_libro("FILO", "El Capital. Libro II. Tomo II", "Karl Marx")
biblio.agregar_libro("FILO", "Karl Marx", "Karl Korsch")
biblio.agregar_libro("FILO", "Fundamentos de filosofía", "A. Rakitov")
biblio.agregar_libro("FILO", "Sobre Marx y Marxismo", "Manuel Sacristán")
biblio.agregar_libro("FILO", "¿Qué es la vida?", "Erwin Schrödinger")
biblio.agregar_libro("FILO", "El sistema de clasificación de Linneo", "David Alvargonzález")
biblio.agregar_libro("FILO", "Filosofía de la Cosmología", "Carlos M. Massini-Correas")
biblio.agregar_libro("FILO", "La relatividad ontológica", "W.V. Quine")
biblio.agregar_libro("FILO", "El gen egoísta", "Richard Dawkins")
biblio.agregar_libro("FILO", "Oriente y Occidente", "Luis Racionero")
biblio.agregar_libro("FILO", "Amor y pedagogía", "Miguel de Unamuno")
biblio.agregar_libro("FILO", "Niebla", "Miguel de Unamuno")
biblio.agregar_libro("FILO", "El positivismo lógico", "A. J. Ayer")
biblio.agregar_libro("FILO", "Tractatus Logico-Philosophicus", "Ludwig Wittgenstein")
biblio.agregar_libro("FILO", "El Capital. Libro I. Tomo I", "Karl Marx")
biblio.agregar_libro("FILO", "El Capital. Libro I. Tomo II", "Karl Marx")
biblio.agregar_libro("FILO", "El Capital. Libro I. Tomo III", "Karl Marx")
biblio.agregar_libro("FILO", "El Capital. Libro II. Tomo I", "Karl Marx")
biblio.agregar_libro("FILO", "El Capital. Libro II. Tomo II", "Karl Marx")
biblio.agregar_libro("FILO", "Karl Marx", "Karl Korsch")
biblio.agregar_libro("FILO", "Fundamentos de filosofía", "A. Rakitov")
biblio.agregar_libro("FILO", "Sobre Marx y Marxismo", "Manuel Sacristán")
biblio.agregar_libro("FILO", "¿Qué es la vida?", "Erwin Schrödinger")
biblio.agregar_libro("FILO", "El sistema de clasificación de Linneo", "David Alvargonzález")
biblio.agregar_libro("FILO", "Filosofía de la Cosmología", "Carlos M. Massini-Correas")
biblio.agregar_libro("FILO", "La relatividad ontológica", "W.V. Quine")
biblio.agregar_libro("FILO", "El gen egoísta", "Richard Dawkins")
biblio.agregar_libro("FILO", "Oriente y Occidente", "Luis Racionero")
biblio.agregar_libro("FILO", "Amor y pedagogía", "Miguel de Unamuno")
biblio.agregar_libro("FILO", "Niebla", "Miguel de Unamuno")
biblio.agregar_libro("FILO", "El positivismo lógico", "A. J. Ayer")
biblio.agregar_libro("FILO", "Tractatus Logico-Philosophicus", "Ludwig Wittgenstein")
biblio.agregar_libro("FILO", "El Ego Trascendental", "Gustavo Bueno")
biblio.agregar_libro("FILO", "¿Qué es la filosofía?", "Gustavo Bueno")
biblio.agregar_libro("FILO", "Einstein. Historia de un espíritu", "Desiderio Papp")
biblio.agregar_libro("FILO", "Principios actuales y fundamentales de la Filosofía", "G. Politzer")
biblio.agregar_libro("FILO", "Confucio, los cuatro libros clásicos", "Montseant Martí i Bugués")
biblio.agregar_libro("FILO", "Filósofos ante Cristo", "Enrique Bonete Perales")
biblio.agregar_libro("FILO", "La física de Dios", "Joseph Selbie")
biblio.agregar_libro("FILO", "Historia del mundo contada para escépticos", "Juan Eslava Galán")
biblio.agregar_libro("FILO", "La mente del simio", "David Premack y Ann Premack")
biblio.agregar_libro("FILO", "Meditaciones sobre la literatura y el arte", "José Ortega y Gasset")
biblio.agregar_libro("FILO", "La rebelión de las masas", "José Ortega y Gasset")
biblio.agregar_libro("FILO", "Del sentimiento trágico de la vida / La agonía del cristianismo", "Miguel de Unamuno")
biblio.agregar_libro("FILO", "Ser y tiempo", "Martin Heidegger")
biblio.agregar_libro("FILO", "Lo bello y lo siniestro", "Eugenio Trías")
biblio.agregar_libro("FILO", "El canto de las sirenas", "Eugenio Trías")
biblio.agregar_libro("FILO", "La mente humana", "José Luis Pinillos")
biblio.agregar_libro("FILO", "El pensamiento utópico contemporáneo", "Antonio Monclús")
biblio.agregar_libro("FILO", "Cómo dominar el tiempo", "Jean-Louis Servan-Schreiber")
biblio.agregar_libro("FILO", "El vendedor de tiempo", "Fernando Trías de Bes")
biblio.agregar_libro("FILO", "El mundo de Sofía", "Jostein Gaarder")
biblio.agregar_libro("FILO", "Historia general de la Filosofía", "Alberto Schwegler")
biblio.agregar_libro("FILO", "La Historia de la Filosofía en la Enciclopedia. Volumen I", "Diderot")
biblio.agregar_libro("FILO", "La Historia de la Filosofía en la Enciclopedia. Volumen II", "Diderot")
biblio.agregar_libro("FILO", "Historia de la Filosofía", "Julián Marías")
biblio.agregar_libro("FILO", "El miedo a la libertad", "Erich Fromm")

# ==== FÍSICA / ASTRONOMÍA ====
biblio.agregar_libro("FISI", "Hacia las estrellas", "Alex Riveiro", "Astronomía")
biblio.agregar_libro("FISI", "Astrofísica para gente con prisas", "Neil deGrasse Tyson", "Astronomía")
biblio.agregar_libro("FISI", "Nueva astronomía recreativa", "V.P. Komarov", "Astronomía")
biblio.agregar_libro("FISI", "Enciclopedia visual de las preguntas: El universo", "El País", "Astronomía")
biblio.agregar_libro("FISI", "Abecedario cósmico", "V. Gorkov", "Astronomía")
biblio.agregar_libro("FISI", "Cosmos", "Carl Sagan", "Astronomía")
biblio.agregar_libro("FISI", "Un punto azul pálido", "Carl Sagan", "Astronomía")
biblio.agregar_libro("FISI", "Galaxia", "National Geographic", "Astronomía")
biblio.agregar_libro("FISI", "Atlas arqueoastronomía", "Belmonte; Hoskin", "Astronomía")
biblio.agregar_libro("FISI", "La astronomía en el antiguo testamento", "Juan V. Schiaparelli", "Astronomía")
biblio.agregar_libro("FISI", "Grandes astrónomos anteriores a Newton", "Francisco Ango", "Astronomía")
biblio.agregar_libro("FISI", "Guía del cielo", "No aplicable", "Astronomía")
biblio.agregar_libro("FISI", "Libro de las estrellas en tres dimensiones", "No aplicable", "Astronomía")
biblio.agregar_libro("FISI", "Atlas de astronomía", "Alliance Atlas", "Astronomía")
biblio.agregar_libro("FISI", "Anuario del observatorio astronómico año 2000", "Ministerio de Fomento IGM", "Astronomía")
biblio.agregar_libro("FISI", "Estrellas, ilustraciones y gráficas", "Guías de naturaleza Blume", "Astronomía")

# ==== FÍSICA / GENERAL ====
biblio.agregar_libro("FISI", "Física al alcance de todos: La temperatura", "Editorial Mir Moscú", "General")
biblio.agregar_libro("FISI", "Física al alcance de todos: Paradojas y sofismas físicos", "Editorial Mir Moscú", "General")
biblio.agregar_libro("FISI", "Física al alcance de todos: Ondas a nuestro alrededor", "Editorial Mir Moscú", "General")
biblio.agregar_libro("FISI", "Física al alcance de todos: Física y geodinámica de la atmósfera", "Editorial Mir Moscú", "General")
biblio.agregar_libro("FISI", "Física al alcance de todos: Conceptos físicos universales", "Editorial Mir Moscú", "General")
biblio.agregar_libro("FISI", "Física al alcance de todos: El mundo relativista", "Editorial Mir Moscú", "General")
biblio.agregar_libro("FISI", "Física al alcance de todos: Nuestro planeta la Tierra", "Editorial Mir Moscú", "General")
biblio.agregar_libro("FISI", "Física al alcance de todos: La naturaleza del magnetismo", "Editorial Mir Moscú", "General")
biblio.agregar_libro("FISI", "Física al alcance de todos: La caza del cero absoluto", "Editorial Mir Moscú", "General")
biblio.agregar_libro("FISI", "El divertido juego de la física", "Y. Perlmann", "General")
biblio.agregar_libro("FISI", "Problemas experimentales ingeniosos de Física", "Editorial Mir Moscú", "General")

# ==== FÍSICA / TEÓRICA ====
biblio.agregar_libro("FISI", "El tejido del cosmos", "Brian Greene", "Teórica")
biblio.agregar_libro("FISI", "Las ondas del espacio-tiempo: la revolución de las ondas gravitatorias", "Matteo Barsuglia", "Teórica")
biblio.agregar_libro("FISI", "Un paseo por el cosmos: La materia oscura", "Un paseo por el cosmos", "Teórica")
biblio.agregar_libro("FISI", "Un paseo por el cosmos: Espacio-tiempo cuántico", "Un paseo por el cosmos", "Teórica")
biblio.agregar_libro("FISI", "Historia del tiempo", "Stephen W. Hawking", "Teórica")
biblio.agregar_libro("FISI", "¿Qué sabemos del universo?", "Juan Pérez Mercader", "Teórica")
biblio.agregar_libro("FISI", "El bosón de Higgs", "Alberto Casas y Teresa Rodrigo CSIC", "Teórica")
biblio.agregar_libro("FISI", "El LHC y la frontera de la física", "Alberto Casas", "Teórica")
biblio.agregar_libro("FISI", "¿Y si el tiempo no existiera?", "Carlo Rovelli", "Teórica")

# ==== GEOGRAFÍA / MAPAS ====
biblio.agregar_libro("GEOG", "Gran atlas Salvat universal - Europa y planisferios", "Salvat", "Mapas")
biblio.agregar_libro("GEOG", "Gran atlas Salvat universal - Asia y África", "Salvat", "Mapas")
biblio.agregar_libro("GEOG", "Gran atlas Salvat universal - América del Norte y América del Sur", "Salvat", "Mapas")
biblio.agregar_libro("GEOG", "Gran atlas Salvat universal - América del Sur, Oceanía e Índice", "Salvat", "Mapas")
biblio.agregar_libro("GEOG", "Atlas de España", "El Pais - Aguilar", "Mapas")
biblio.agregar_libro("GEOG", "Atlas de España II", "El Pais - Aguilar", "Mapas")
biblio.agregar_libro("GEOG", "Mapas. Ciudades con encanto: París, LOndres, Nueva York, Roma, Praga, Viena", "No aplicable", "Mapas")

# ==== GEOGRAFÍA / VIAJES ====
biblio.agregar_libro("GEOG", "Guía, Madeira y Porto Santo", "Sergio Fonseca y Susana Fonseca", "Viajes")
biblio.agregar_libro("GEOG", "Mérida, guía de viaje 2024", "Bella Snow", "Viajes")
biblio.agregar_libro("GEOG", "Guía, Noruega", "No aplicable", "Viajes")
biblio.agregar_libro("GEOG", "Guía, Los pueblos de La Val d'Aran", "No aplicable", "Viajes")
biblio.agregar_libro("GEOG", "Guía, A pie por Roma", "No aplicable", "Viajes")
biblio.agregar_libro("GEOG", "Guía, La sagrada Meteora", "No aplicable", "Viajes")
biblio.agregar_libro("GEOG", "Guía, Chipre", "No aplicable", "Viajes")
biblio.agregar_libro("GEOG", "Guía Viajes Roma - National Geographic", "No aplicable", "Viajes")
biblio.agregar_libro("GEOG", "Postales Japón", "No aplicable", "Viajes")
biblio.agregar_libro("GEOG", "Bergen", "No aplicable", "Viajes")
biblio.agregar_libro("GEOG", "Velkommen", "Noruega", "Viajes")
biblio.agregar_libro("GEOG", "Castillos del Loira", "Valoire", "Viajes")

# ==== HISTORIA ====
biblio.agregar_libro("HIST", "Historia de Grecia", "Pastora Barahona")
biblio.agregar_libro("HIST", "Gran Historia Universal, el Mundo Griego", "Club Internacional del Libro de Madrid")
biblio.agregar_libro("HIST", "Historia de Madrid", "Antonio Fernández García")
biblio.agregar_libro("HIST", "Atlas ilustrado de la España Romana", "Susaeta")
biblio.agregar_libro("HIST", "La España Visigoda", "Susaeta")
biblio.agregar_libro("HIST", "Desde Don Pelayo hasta Don Juan Carlos I", "Pedro de Felipe del Rey")
biblio.agregar_libro("HIST", "Cartas de la Conquista de México", "Hernán Cortés")
biblio.agregar_libro("HIST", "Historia de la Guerra del Peloponeso", "Tucídides")
biblio.agregar_libro("HIST", "SPQR, una historia de la Antigua Roma", "Mary Beard")
biblio.agregar_libro("HIST", "La muerte del Che Guevara", "Jay Cantor")
biblio.agregar_libro("HIST", "Los masones", "César Vidal")
biblio.agregar_libro("HIST", "La España de Fernando VII", "Miguel Artola")
biblio.agregar_libro("HIST", "Monarquía e Imperio, el Reinado de Carlos V", "John Lynch")
biblio.agregar_libro("HIST", "Sapiens, de animales a dioses", "Yuval Noah Harari")
biblio.agregar_libro("HIST", "Conversaciones con Faustino Cordon", "Antonio Núñez")
biblio.agregar_libro("HIST", "Breve historia del arte china", "Gina Pischel")
biblio.agregar_libro("HIST", "1808, el dos de mayo, tres miradas", "Alejo Carpentier")
biblio.agregar_libro("HIST", "El Egipto del imperio antiguo", "Josep Padró Parcersa")
biblio.agregar_libro("HIST", "Mi vida", "León Trotsky")
biblio.agregar_libro("HIST", "El mundo de ayer, memorias de un europeo", "Stefan Zweig")
biblio.agregar_libro("HIST", "España en la vida italiana durante el Renacimiento", "Benedetto Croce")
biblio.agregar_libro("HIST", "La España elegida", "Juan Luis Arsuaga")
biblio.agregar_libro("HIST", "Los cristianos derrotados", "Antonio Piñero")
biblio.agregar_libro("HIST", "La sociedad japonesa", "Andrés Bellersart")
biblio.agregar_libro("HIST", "Siglo y medio de fósforos", "Fosforera Española")
biblio.agregar_libro("HIST", "La formación de la humanidad", "Richard Leakey")
biblio.agregar_libro("HIST", "Tres retratos de la Revolución Francesa", "Albert Z. Manfred")
biblio.agregar_libro("HIST", "Se consagra a Alexander Pushkin", "L.S.")
biblio.agregar_libro("HIST", "Los protocolos de los sabios de Sion", "Editorial Época")
biblio.agregar_libro("HIST", "Los doce Césares", "Suetonio")
biblio.agregar_libro("HIST", "Magallanes", "Stefan Zweig")
biblio.agregar_libro("HIST", "Acerca del anarquismo y el anarcorsindicalismo", "Marx, Engels, Lenin")
biblio.agregar_libro("HIST", "Lenin", "Herman Weber")
biblio.agregar_libro("HIST", "Mao Zedong", "Freeman Gamm")
biblio.agregar_libro("HIST", "Bolívar", "Jorge Campos")
biblio.agregar_libro("HIST", "La caída de los dioses", "Daniel Solars")
biblio.agregar_libro("HIST", "Alejandro Magno", "Nicholas Nicastro")
biblio.agregar_libro("HIST", "Hacia la casa común europea", "Mijail Gorbachov")
biblio.agregar_libro("HIST", "Enigmas de la historia", "Editorial Progreso")
biblio.agregar_libro("HIST", "V. I. Lenin, esbozo biográfico", "Editorial Progreso")
biblio.agregar_libro("HIST", "Historia de América Latina", "Carlos M.Rama")
biblio.agregar_libro("HIST", "Usos y costumbres sexuales de los Reyes de España", "Emilio Calderón")
biblio.agregar_libro("HIST", "Beethoven", "Hector Berlioz")
biblio.agregar_libro("HIST", "La vida heroica del Marie Curie", "Eva Curie")
biblio.agregar_libro("HIST", "Historia del Arte del Próximo Oriente", "Fernando de Olaguer-Feliu")
biblio.agregar_libro("HIST", "Historia del Arte de la Baja Edad Media", "Mª Ángeles B. Piquero")
biblio.agregar_libro("HIST", "La vida cotidiana en la España Musulmana", "Fernando Pati-Playá")
biblio.agregar_libro("HIST", "De Gaulle", "François Mauriac")
biblio.agregar_libro("HIST", "Cuaderno de notas", "Leonardo Da Vinci")
biblio.agregar_libro("HIST", "El mundo de las antigüedades: cerámica del siglo XVIII", "Planeta-Agostini")
biblio.agregar_libro("HIST", "El mundo de las antigüedades: cerámica del siglo XIX", "Planeta-Agostini")
biblio.agregar_libro("HIST", "El mundo de las antigüedades: el mueble del siglo XX", "Planeta-Agostini")
biblio.agregar_libro("HIST", "El mundo de las antigüedades: tapices de los siglos XV, XVI, XVII", "Planeta-Agostini")
biblio.agregar_libro("HIST", "El mundo de las antigüedades: tapices del siglo XVIII", "Planeta-Agostini")
biblio.agregar_libro("HIST", "El mundo de las antigüedades: plata del siglo XVIII", "Planeta-Agostini")
biblio.agregar_libro("HIST", "El mundo de las antigüedades: plata del siglos XIX", "Planeta-Agostini")
biblio.agregar_libro("HIST", "El mundo de las antigüedades: vidrio de los siglos XV, XVI, XVII", "Planeta-Agostini")
biblio.agregar_libro("HIST", "El mundo de las antigüedades: vidrio de los siglos XVIII y XIX", "Planeta-Agostini")
biblio.agregar_libro("HIST", "Colecciones de arqueología y etnología de América", "Planeta-Agostini")
biblio.agregar_libro("HIST", "El mundo de las antigüedades: alfombras orientales", "Planeta-Agostini")
biblio.agregar_libro("HIST", "Porcelana", "Susana Morgan")
biblio.agregar_libro("HIST", "Los nueve libros de la historia", "Heródoto")
biblio.agregar_libro("HIST", "Obras", "Don Gaspar Melchor Jovellanos")
biblio.agregar_libro("HIST", "Beethoven", "Erik Luguet")
biblio.agregar_libro("HIST", "Mi infancia y juventud", "Santiago Ramón y Cajal")
biblio.agregar_libro("HIST", "La obra se Valle-Inclán", "Fernando de Toro-Garland")
biblio.agregar_libro("HIST", "Vivir para contarla", "Gabriel García Márquez")
biblio.agregar_libro("HIST", "Carta al padre", "Franz Kafka")
biblio.agregar_libro("HIST", "El pequeño mundo del hombre", "Francisco Rico")

# ==== IDIOMAS ====
biblio.agregar_libro("IDIO", "English File A1 Student's book", "OXFORD")
biblio.agregar_libro("IDIO", "English File A1 Workbook", "OXFORD")
biblio.agregar_libro("IDIO", "Move On Student's book", "OXFORD")
biblio.agregar_libro("IDIO", "Move On Workbook", "OXFORD")
biblio.agregar_libro("IDIO", "View Points for Bachillerato", "OXFORD")
biblio.agregar_libro("IDIO", "That's English Module 5", "Burlington Books")
biblio.agregar_libro("IDIO", "That's English Module 6", "Burlington Books")
biblio.agregar_libro("IDIO", "Compact Advanced Student's book", "Cambridge")
biblio.agregar_libro("IDIO", "KEK Net pocket Student", "National Geographic")
biblio.agregar_libro("IDIO", "Proficiency testbuilder", "Mcmillan")
biblio.agregar_libro("IDIO", "Diccionario Cambridge Compact English-Spanish", "Cambridge")
biblio.agregar_libro("IDIO", "Diccionario para principiantes Inglés", "Susaeta")
biblio.agregar_libro("IDIO", "Diccionario Inglés-Español", "Geddes & Grosset")
biblio.agregar_libro("IDIO", "Diccionario visual Inglés-Español", "Saldaña")
biblio.agregar_libro("IDIO", "Diccionario Inglés", "VOX")
biblio.agregar_libro("IDIO", "Diccionario Inglés-Español", "Langenscheidt")
biblio.agregar_libro("IDIO", "Advances Grammar in Use", "Cambridge")
biblio.agregar_libro("IDIO", "English Vocabulary in Use", "Cambridge")
biblio.agregar_libro("IDIO", "Inglés Curso Intensivo", "No aplicable")
biblio.agregar_libro("IDIO", "The Adventures of Tom Sawyer", "Mark Twain")
biblio.agregar_libro("IDIO", "Grammaire Progressive du Français B2-C2", "CLE International")
biblio.agregar_libro("IDIO", "Alter Ego, C1-B2", "Hachette")
biblio.agregar_libro("IDIO", "Grammaire Progressive du Français", "CLE International")
biblio.agregar_libro("IDIO", "Vocabulaire Progressif du Français", "CLE International")
biblio.agregar_libro("IDIO", "Diccionario Español-Francés", "Collins")
biblio.agregar_libro("IDIO", "L'ocean est mon frère", "Collins")
biblio.agregar_libro("IDIO", "La beauté naturelle", "Carole Dichampt")
biblio.agregar_libro("IDIO", "Remi et Juliette", "Léo Lamarche")
biblio.agregar_libro("IDIO", "Le fils de Nasreddine", "Brigitte Arnaudiès")
biblio.agregar_libro("IDIO", "Le partage des oies", "Brigitte Arnaudiès")
biblio.agregar_libro("IDIO", "Pierrot", "Annette Maxime")
biblio.agregar_libro("IDIO", "Enfermés", "Annette Maxime")
biblio.agregar_libro("IDIO", "¿Quiere usted saber alemán en 10 días?", "Métodos Robertson")
biblio.agregar_libro("IDIO", "Diccionario Alemán-Español", "VOX")
biblio.agregar_libro("IDIO", "Diccionario visual Alemán-Español", "Saldaña")
biblio.agregar_libro("IDIO", "Curso completo de lengua rusa, nivel básico", "Ediciones hispano eslavas")
biblio.agregar_libro("IDIO", "Curso completo de lengua rusa, nivel intermedio y avanzado", "Ediciones hispano eslavas")
biblio.agregar_libro("IDIO", "Diccionario Ruso-Español", "Marzishevskaya")
biblio.agregar_libro("IDIO", "Complete norweigian", "Teach yourself")
biblio.agregar_libro("IDIO", "Lengua latina, pars I", "Hans H.Orberg")
biblio.agregar_libro("IDIO", "Lengua latina, exercitia latina I", "Hans H.Orberg")
biblio.agregar_libro("IDIO", "Diccionario Latín-Español", "VOX")
biblio.agregar_libro("IDIO", "Antología latina", "Editorial Gredos")
biblio.agregar_libro("IDIO", "Diccionario Esencial Latín-Español", "VOX")


# ==== JUVENIL ====
biblio.agregar_libro("JUVE", "Super Humor Mortadelo y Filemón", "Ed. Ediciones B", "Cómics")
biblio.agregar_libro("JUVE", "Super Humor Mortadelo y Filemón nº 42", "Ed. Ediciones B", "Cómics")
biblio.agregar_libro("JUVE", "El aniversario de Astérix y Obélix, el libro de oro", "Ed. Salvat", "Cómics")
biblio.agregar_libro("JUVE", "Astérix en la India", "Ed. Salvat", "Cómics")
biblio.agregar_libro("JUVE", "Mortadelo y Filemón Mundial 2010", "Ed. Ediciones B", "Cómics")
biblio.agregar_libro("JUVE", "Mortadelo y Filemón, álbum de coleccionista", "Ed. Salvat", "Cómics")
biblio.agregar_libro("JUVE", "Mortadelo y Filemón, euro basket 2001", "Ed. Olé", "Cómics")
biblio.agregar_libro("JUVE", "Mortadelo Top Cómic nº29", "Ed. Ediciones B", "Cómics")

biblio.agregar_libro("JUVE", "Aventuras de papel", "Ken Blackburn", "Conocimiento")
biblio.agregar_libro("JUVE", "Las 323 adivinanzas más famosas", "Ed. Susaeta", "Conocimiento")
biblio.agregar_libro("JUVE", "Ajedrez para niños, ¡A jugar ya! 1", "Carlos García Palermo", "Conocimiento")
biblio.agregar_libro("JUVE", "Ajedrez para niños, ¡A jugar ya! 2", "Carlos García Palermo", "Conocimiento")
biblio.agregar_libro("JUVE", "Parajopelícula", "P. Chicharro Valencia", "Conocimiento")
biblio.agregar_libro("JUVE", "Libro de juegos", "K. Snewmay", "Conocimiento")
biblio.agregar_libro("JUVE", "Adivinanzas, pictogramas para niños", "Ed. Divulga", "Conocimiento")
biblio.agregar_libro("JUVE", "Poemas y cuentos, pictogramas para niños", "Ed. Divulga", "Conocimiento")
biblio.agregar_libro("JUVE", "Los experimentos de Flipy", "Ed. Aguilar", "Conocimiento")
biblio.agregar_libro("JUVE", "Hay historias, mañana sol", "Sabine Seyffert", "Conocimiento")
biblio.agregar_libro("JUVE", "Los supermimosos", "Ed. Brab", "Conocimiento")
biblio.agregar_libro("JUVE", "Las claves del amor", "Ed. Duval Fontana", "Conocimiento")
biblio.agregar_libro("JUVE", "Atlas Mundial", "Ed. LTBASA", "Conocimiento")
biblio.agregar_libro("JUVE", "Historia Mundial", "Ed. LTBASA", "Conocimiento")
biblio.agregar_libro("JUVE", "El mundo natural", "Ed. LTBASA", "Conocimiento")
biblio.agregar_libro("JUVE", "Mi primer gran enciclopedia", "Ed. Parragon", "Conocimiento")
biblio.agregar_libro("JUVE", "El libro de la ciencia", "Ed. Parragon", "Conocimiento")
biblio.agregar_libro("JUVE", "La enciclopedia de los conocimientos", "Ed. Océano", "Conocimiento")
biblio.agregar_libro("JUVE", "Mi primer libro de grandes exploradores", "Ed. Parragon", "Conocimiento")
biblio.agregar_libro("JUVE", "Cómo poner mil papel en el agua", "Ed. SM", "Conocimiento")
biblio.agregar_libro("JUVE", "Mi mundo de día y de noche", "Ed. SM", "Conocimiento")
biblio.agregar_libro("JUVE", "Máquinas asombrosas", "Ed. Parragon", "Conocimiento")
biblio.agregar_libro("JUVE", "Atlas del mundo", "Ed. Parragon", "Conocimiento")
biblio.agregar_libro("JUVE", "Rafael Alberti para niños", "Ed. Susaeta", "Conocimiento")
biblio.agregar_libro("JUVE", "Aprende Ballet", "Usborne", "Conocimiento")
biblio.agregar_libro("JUVE", "Aventuras, robots para niños", "Ed. Divulga", "Conocimiento")
biblio.agregar_libro("JUVE", "Aventuras, planetas para niños", "Ed. Divulga", "Conocimiento")
biblio.agregar_libro("JUVE", "Spiel rein Abenteuer", "Ed. Ravensburger", "Conocimiento")
biblio.agregar_libro("JUVE", "Astronomía de niños", "Pedro Gañedos", "Conocimiento")
biblio.agregar_libro("JUVE", "Los animales y su mundo", "Ed. Todolibro", "Conocimiento")
biblio.agregar_libro("JUVE", "Los animales y su mundo, montañas y bosques", "Ed. Todolibro", "Conocimiento")
biblio.agregar_libro("JUVE", "Los porqués de los animales", "Ed. Todolibro", "Conocimiento")

biblio.agregar_libro("JUVE", "Antoñita la fantástica y su tía Carol", "Borita Casas", "Novela")
biblio.agregar_libro("JUVE", "Antoñita la fantástica de vacaciones", "Borita Casas", "Novela")
biblio.agregar_libro("JUVE", "Pinner curso en torres de Malory", "Enid Blyton", "Novela")
biblio.agregar_libro("JUVE", "La fábrica de nubes", "Jordi Sierra i Fabra", "Novela")
biblio.agregar_libro("JUVE", "Breves y eternos cuentos de luchas y poesía", "Ed. escritores", "Novela")
biblio.agregar_libro("JUVE", "El misterio de los estudios Kellerman", "Ken Follett", "Novela")
biblio.agregar_libro("JUVE", "No pidas sardina fuera de temporada", "Andreu Martin", "Novela")
biblio.agregar_libro("JUVE", "Viaje en el tiempo", "Geronimo Stilton", "Novela")
biblio.agregar_libro("JUVE", "El fantasma de Canterville", "Wilde", "Novela")
biblio.agregar_libro("JUVE", "El principito, con ilustraciones del autor", "Antoine de Saint-Exupéry", "Novela")
biblio.agregar_libro("JUVE", "Árboles y rimas para niños", "Ed. Parragon", "Novela")
biblio.agregar_libro("JUVE", "El caballero del torre", "Anónimo", "Novela")
biblio.agregar_libro("JUVE", "Las lágrimas de Shiva", "César Mallorquí", "Novela")
biblio.agregar_libro("JUVE", "El valle de los trolls", "Ed. Rolf Lidberg", "Novela")
biblio.agregar_libro("JUVE", "Lily Quench y las montañas negras", "Natalie Jane Prior", "Novela")
biblio.agregar_libro("JUVE", "Las aventuras del Doctor Dolittle", "Hugh Lofting", "Novela")
biblio.agregar_libro("JUVE", "Cuentos chinos", "J.H. Sweet", "Novela")
biblio.agregar_libro("JUVE", "Las crónicas de Spiderwick", "Diterlizzi & Black", "Novela")
biblio.agregar_libro("JUVE", "La leyenda de Sleepy Hollow y otros cuentos fantásticos", "Ed. Oxford", "Novela")
biblio.agregar_libro("JUVE", "Los tres naranjas del amor y otros cuentos en español", "Carmen Bravo Villasante", "Novela")
biblio.agregar_libro("JUVE", "Lily y la pantera misiñosa", "Kristin Larsen", "Novela")
biblio.agregar_libro("JUVE", "Dos preguntas y respuestas de Gloria", "Gloria Fuertes", "Novela")
biblio.agregar_libro("JUVE", "Matilda", "Roald Dahl", "Novela")
biblio.agregar_libro("JUVE", "El patito feo, un cuento de Valdemar Daae", "C. Andersen", "Novela")
biblio.agregar_libro("JUVE", "Historia de un segundo", "Jordi Sierra i Fabra", "Novela")
biblio.agregar_libro("JUVE", "Alice in Wonderland", "Lewis Carroll", "Novela")
biblio.agregar_libro("JUVE", "Abbadat, el profesor, la marioneta y el profeta", "Editorial Planeta Junior", "Novela")
biblio.agregar_libro("JUVE", "Fábulas Chinas", "Ediciones Literatura Universal", "Novela")
biblio.agregar_libro("JUVE", "Antoñita la fantástica cumple diez años", "Borita Casas", "Novela")
biblio.agregar_libro("JUVE", "El principito", "Ed. Enrique Sancho Editor", "Novela")
biblio.agregar_libro("JUVE", "Cuentos del profesor ruso", "Editorial Progreso", "Novela")
biblio.agregar_libro("JUVE", "El misterio de la isla de Tökland", "Joan Manuel Gisbert", "Novela")
biblio.agregar_libro("JUVE", "La casa de los espejos", "Ulysses Moore", "Novela")
biblio.agregar_libro("JUVE", "Bussi la hermana", "Ed. Raduga", "Novela")
biblio.agregar_libro("JUVE", "Lily Quench y el dragón de Ashby", "Natalie Jane Prior", "Novela")
biblio.agregar_libro("JUVE", "Jonathan Livingston. Seguel Gaviota", "Richard Bach", "Novela")
biblio.agregar_libro("JUVE", "Jonas el hijo de los damnificados", "Joachim Friedrich", "Novela")
biblio.agregar_libro("JUVE", "Fábulas hindúes", "Ed. Astra", "Novela")
biblio.agregar_libro("JUVE", "El lago de los cisnes", "Juana Spyras", "Novela")
biblio.agregar_libro("JUVE", "The eye of childhood", "Ed. Oxford", "Novela")

# ==== MITOLOGÍA ====
biblio.agregar_libro("MITO", "Gilgamesh", "Poema Babilonio")
biblio.agregar_libro("MITO", "Ilíada", "Homero")
biblio.agregar_libro("MITO", "Odisea", "Homero")
biblio.agregar_libro("MITO", "Tragedias completas", "Esquilo")
biblio.agregar_libro("MITO", "Tragedias completas", "Sófocles")
biblio.agregar_libro("MITO", "Tragedias", "Eurípides")
biblio.agregar_libro("MITO", "Las troyanas", "Eurípides")
biblio.agregar_libro("MITO", "Las nubes, Lisístrata, Dinero", "Aristófanes")
biblio.agregar_libro("MITO", "Zeus conquista el Olimpo", "No aplicable")
biblio.agregar_libro("MITO", "Mitología clásica", "M.J. Almagro Gorbea")
biblio.agregar_libro("MITO", "Hécate, La diosa sombría", "Mario Agudo Villanueva")
biblio.agregar_libro("MITO", "Eneida", "Virgilio")
biblio.agregar_libro("MITO", "Metamorfosis", "Ovidio")
biblio.agregar_libro("MITO", "El viaje de los argonautas", "Apolonio de Rodas")
biblio.agregar_libro("MITO", "Divina Comedia, Editorial Alianza", "Dante Alighieri")
biblio.agregar_libro("MITO", "Divina Comedia, Editorial Cátedra", "Dante Alighieri")
biblio.agregar_libro("MITO", "Divina Comedia, Las 135 ilustraciones de Gustavo Doré", "Gustavo Doré")
biblio.agregar_libro("MITO", "El paraíso perdido", "John Milton")
biblio.agregar_libro("MITO", "Borges esencial", "Jorge Luis Borges")
biblio.agregar_libro("MITO", "Las bodas de Cadmo y Harmonía", "Roberto Calasso")
biblio.agregar_libro("MITO", "Érase una vez... El universo, los dioses, los hombres", "Jean Pierre Vernant")
biblio.agregar_libro("MITO", "El arte de amar", "Ovidio")
biblio.agregar_libro("MITO", "El gran libro de la mitología griega", "Robin Hart")

# ==== NOVELA /  ====
biblio.agregar_libro("NOVE", "Don Quijote de la Mancha, ilustrado por Doré", "Miguel de Cervantes", "Aventuras")
biblio.agregar_libro("NOVE", "El ingenioso hidalgo Don Quijote de la Mancha", "Miguel de Cervantes", "Aventuras")
biblio.agregar_libro("NOVE", "El país de los Kenders", "Mary Kirchoff", "Aventuras")
biblio.agregar_libro("NOVE", "Territorio Comanche", "Arturo Pérez Reverte", "Aventuras")
biblio.agregar_libro("NOVE", "Dos capitanes, dos rumbos", "Veniamin Kaverin", "Aventuras")
biblio.agregar_libro("NOVE", "El señor de los ladrones", "Cornelia Funke", "Aventuras")
biblio.agregar_libro("NOVE", "Las nieves del Kilimanjaro", "Ernest Hemingway", "Aventuras")
biblio.agregar_libro("NOVE", "Noche en Bombay", "Louis Bromfield", "Aventuras")
biblio.agregar_libro("NOVE", "El corsario negro", "Emilio Salgari", "Aventuras")
biblio.agregar_libro("NOVE", "Moby Dick", "Herman Melville", "Aventuras")
biblio.agregar_libro("NOVE", "La quimera de oro", "Jack London", "Aventuras")
biblio.agregar_libro("NOVE", "Obras completas. Tomo I", "Julio Verne", "Aventuras")
biblio.agregar_libro("NOVE", "Obras completas. Tomo II", "Julio Verne", "Aventuras")
biblio.agregar_libro("NOVE", "Obras completas. Tomo III", "Julio Verne", "Aventuras")
biblio.agregar_libro("NOVE", "Obras completas. Tomo IV", "Julio Verne", "Aventuras")
biblio.agregar_libro("NOVE", "Obras completas. Tomo V", "Julio Verne", "Aventuras")
biblio.agregar_libro("NOVE", "Obras completas. Tomo VI", "Julio Verne", "Aventuras")
biblio.agregar_libro("NOVE", "Obras completas. Tomo VII", "Julio Verne", "Aventuras")
biblio.agregar_libro("NOVE", "Obras completas. Tomo VII", "Julio Verne", "Aventuras")

biblio.agregar_libro("NOVE", "Mar fuerte que el último", "Gueorgui Bernátzko", "Bélica")
biblio.agregar_libro("NOVE", "Tiempo de prueba", "Pavel Nilov", "Bélica")
biblio.agregar_libro("NOVE", "De los vivos y de los muertos", "Konstantin Simonov", "Bélica")
biblio.agregar_libro("NOVE", "Balada alpina", "Vasil Bykov", "Bélica")
biblio.agregar_libro("NOVE", "Un día en la vida de Ivan Denisovich", "Aleksandr Solzhenitsyn", "Bélica")
biblio.agregar_libro("NOVE", "Un hombre de verdad", "Boris Polevoy", "Bélica")
biblio.agregar_libro("NOVE", "La joven guardia", "Alexander Fadéiev", "Bélica")

biblio.agregar_libro("NOVE", "Teluria", "Pedro Sánchez Paredes", "Ciencia Ficción")
biblio.agregar_libro("NOVE", "Contact", "Carl Sagan", "Ciencia Ficción")
biblio.agregar_libro("NOVE", "El juego de Ender", "Orson Scott Card", "Ciencia Ficción")
biblio.agregar_libro("NOVE", "La torre oscura", "Stephen King", "Ciencia Ficción")
biblio.agregar_libro("NOVE", "Fahrenheit 451", "Ray Bradbury", "Ciencia Ficción")

biblio.agregar_libro("NOVE", "Cien años de soledad", "Gabriel García Márquez", "Fantasía")
biblio.agregar_libro("NOVE", "Narraciones extraordinarias", "Edgar Allan Poe", "Fantasía")
biblio.agregar_libro("NOVE", "Cuentos y prosas", "Rubén Darío", "Fantasía")
biblio.agregar_libro("NOVE", "De que hablan los animales", "L. L. Stishkovskaya", "Fantasía")
biblio.agregar_libro("NOVE", "Memorias de Idhún I: La resistencia", "Laura Gallego García", "Fantasía")
biblio.agregar_libro("NOVE", "Canguro", "D. H. Lawrence", "Fantasía")
biblio.agregar_libro("NOVE", "Cuento de Navidad - El grillo del hogar", "Charles Dickens", "Fantasía")
biblio.agregar_libro("NOVE", "Cuentos", "E. T. A. Hoffmann", "Fantasía")
biblio.agregar_libro("NOVE", "Las mil y una noches", "Ed. Salvat", "Fantasía")
biblio.agregar_libro("NOVE", "El demonio de Arbenois", "Bernard T.", "Fantasía")
biblio.agregar_libro("NOVE", "La sombra de Dreindar", "Bernard T.", "Fantasía")
biblio.agregar_libro("NOVE", "Cien años de soledad", "Gabriel García Márquez", "Fantasía")
biblio.agregar_libro("NOVE", "La montaña mágica", "Thomas Mann", "Fantasía")

biblio.agregar_libro("NOVE", "El Decamerón", "Boccaccio", "Ficción Social")
biblio.agregar_libro("NOVE", "San Manuel Bueno, mártir", "Miguel de Unamuno", "Ficción Social")
biblio.agregar_libro("NOVE", "Crimen y castigo", "Dostoyevski", "Ficción Social")
biblio.agregar_libro("NOVE", "El jugador", "Dostoyevski", "Ficción Social")
biblio.agregar_libro("NOVE", "Fantasías vascas", "Pío Baroja", "Ficción Social") 
biblio.agregar_libro("NOVE", "Rayuela", "Julio Cortázar", "Ficción Social")
biblio.agregar_libro("NOVE", "La sangre de los otros", "Simone de Beauvoir", "Ficción Social")
biblio.agregar_libro("NOVE", "La verdad de las mentiras (Ensayos sobre la novela)", "Mario Vargas Llosa", "Ficción Social")
biblio.agregar_libro("NOVE", "Nuevos cuentos de las colonias", "Rudyard Kipling", "Ficción Social") 
biblio.agregar_libro("NOVE", "El proceso", "Franz Kafka", "Ficción Social")
biblio.agregar_libro("NOVE", "Humillados y ofendidos", "Dostoyevski", "Ficción Social")
biblio.agregar_libro("NOVE", "Elogio de la locura", "Erasmo de Rotterdam", "Ficción Social")
biblio.agregar_libro("NOVE", "Vinieron las lluvias, I", "Louis Bromfield", "Ficción Social") 
biblio.agregar_libro("NOVE", "De cómo los turcos descubrieron América", "Jorge Amado", "Ficción Social") 
biblio.agregar_libro("NOVE", "Biblioteca fundamental: El lobo estepario", "Hermann Hesse", "Ficción Social")
biblio.agregar_libro("NOVE", "El desafío", "A. I. Kuprin", "Ficción Social") 
biblio.agregar_libro("NOVE", "Una comedia ligera", "Eduardo Mendoza", "Ficción Social") 
biblio.agregar_libro("NOVE", "Cándido", "Voltaire", "Ficción Social")
biblio.agregar_libro("NOVE", "Fantasías de un delegado de Hacienda y otros cuentos", "Leopoldo Alas Clarín", "Ficción Social")
biblio.agregar_libro("NOVE", "El pasajero de ultramar", "José M. Guelbenzu", "Ficción Social") 
biblio.agregar_libro("NOVE", "El rey lear de la estepa", "Turgueniev", "Ficción Social") 
biblio.agregar_libro("NOVE", "Historias de almanaque", "Bertolt Brecht", "Ficción Social")
biblio.agregar_libro("NOVE", "Tiempo de abrazar", "Juan Carlos Onetti", "Ficción Social") 
biblio.agregar_libro("NOVE", "La piel de zapa", "Balzac", "Ficción Social")
biblio.agregar_libro("NOVE", "La metamorfosis", "Kafka", "Ficción Social")
biblio.agregar_libro("NOVE", "Iván el imbécil", "León Tolstoi", "Ficción Social")
biblio.agregar_libro("NOVE", "Leyendas rusas", "María José Llorens Camp", "Ficción Social")
biblio.agregar_libro("NOVE", "Crimen y castigo", "Fiódor Dostoyevski", "Ficción Social")
biblio.agregar_libro("NOVE", "Dejemos hablar al viento", "Juan Carlos Onetti", "Ficción Social")
biblio.agregar_libro("NOVE", "Vuelva usted mañana y otros artículos", "Mariano José de Larra", "Ficción Social")
biblio.agregar_libro("NOVE", "Las tribulaciones del estudiante Törless", "Robert Musil", "Ficción Social")
biblio.agregar_libro("NOVE", "Balada de otoño", "Manuel Vicent", "Ficción Social")
biblio.agregar_libro("NOVE", "Sonata de primavera", "Ramón del Valle-Inclán", "Ficción Social")
biblio.agregar_libro("NOVE", "Peón de rey", "Pedro Jesús Fernández", "Ficción Social")
biblio.agregar_libro("NOVE", "Obras completas", "W. Somerset Maugham", "Ficción Social")
biblio.agregar_libro("NOVE", "Contrapunto", "Aldous Huxley", "Ficción Social")
biblio.agregar_libro("NOVE", "Las pequeñas memorias", "José Saramago", "Ficción Social")
biblio.agregar_libro("NOVE", "Alabardas", "José Saramago", "Ficción Social")
biblio.agregar_libro("NOVE", "El nombre duplicado", "José Saramago", "Ficción Social")
biblio.agregar_libro("NOVE", "Los cuadernos de Lanzarote II (1996-1997)", "José Saramago", "Ficción Social")

biblio.agregar_libro("NOVE", "El asesinato de Sócrates", "Marcos Chicot", "Histórica")
biblio.agregar_libro("NOVE", "Las legiones malditas", "Santiago Posteguillo", "Histórica")
biblio.agregar_libro("NOVE", "El faro de Alejandría", "Gillian Bradshaw", "Histórica")
biblio.agregar_libro("NOVE", "La caída de Roma", "Michael Curtis Ford", "Histórica")
biblio.agregar_libro("NOVE", "La hermandad de la Sábana Santa", "Julia Navarro", "Histórica")
biblio.agregar_libro("NOVE", "La sombra del inquisidor", "Roberto Calvo y Marcos Lazzaro", "Histórica")
biblio.agregar_libro("NOVE", "El mercader de libros", "Luis Zueco", "Histórica")
biblio.agregar_libro("NOVE", "El arpista ciego", "Terenci Moix", "Histórica")
biblio.agregar_libro("NOVE", "Las minas del rey Salomón", "H. Rider Haggard", "Histórica")
biblio.agregar_libro("NOVE", "La fiesta del chivo", "Mario Vargas Llosa", "Histórica")
biblio.agregar_libro("NOVE", "Los pilares de la Tierra", "Ken Follett", "Histórica")
biblio.agregar_libro("NOVE", "Un mundo sin fin", "Ken Follett", "Histórica")
biblio.agregar_libro("NOVE", "Una columna de fuego", "Ken Follett", "Histórica")
biblio.agregar_libro("NOVE", "El invierno del mundo", "Ken Follett", "Histórica")
biblio.agregar_libro("NOVE", "Las dos muertes de Mozart", "Joseph Gelinek", "Histórica")
biblio.agregar_libro("NOVE", "El maestro Eckhart", "Juan Bedera", "Histórica")
biblio.agregar_libro("NOVE", "La dama de Urtubi", "Pío Baroja", "Histórica")
biblio.agregar_libro("NOVE", "Retrato de grupo con señora", "Heinrich Böll", "Histórica")
biblio.agregar_libro("NOVE", "Colón", "Pedro Veltes", "Histórica")
biblio.agregar_libro("NOVE", "La cena secreta", "Javier Sierra", "Histórica")
biblio.agregar_libro("NOVE", "El capitán Alatriste", "Arturo Pérez-Reverte", "Histórica")
biblio.agregar_libro("NOVE", "Rusos I", "Edward Rutherfurd", "Histórica")
biblio.agregar_libro("NOVE", "Rusos II", "Edward Rutherfurd", "Histórica")
biblio.agregar_libro("NOVE", "Todo bajo el cielo", "Matilde Asensi", "Histórica")
biblio.agregar_libro("NOVE", "Constantinopla", "Ignacio Tovarey", "Histórica")
biblio.agregar_libro("NOVE", "Archipiélago Gulag", "Aleksandr Solzhenitsyn", "Histórica")
biblio.agregar_libro("NOVE", "Ulises Vol. I", "James Joyce", "Histórica")
biblio.agregar_libro("NOVE", "Ulises Vol. II", "James Joyce", "Histórica")
biblio.agregar_libro("NOVE", "Yo, Claudio", "Robert Graves", "Histórica")
biblio.agregar_libro("NOVE", "El valle de los leones", "Ken Follett", "Histórica")
biblio.agregar_libro("NOVE", "La joven guardia", "Alexander Fadéiev", "Histórica")
biblio.agregar_libro("NOVE", "El largo viaje", "Jorge Semprún", "Histórica")
biblio.agregar_libro("NOVE", "La verdad sobre el caso Savolta", "Eduardo Mendoza", "Histórica")
biblio.agregar_libro("NOVE", "La rosa verde", "Warren A. Silva", "Histórica")
biblio.agregar_libro("NOVE", "El clan del oso cavernario", "Jean M. Auel", "Histórica")
biblio.agregar_libro("NOVE", "Retrato en sepia", "Isabel Allende", "Histórica")
biblio.agregar_libro("NOVE", "El nombre de la rosa", "Umberto Eco", "Histórica")
biblio.agregar_libro("NOVE", "El 19 de marzo y el 1 de mayo", "Benito Pérez Galdós", "Histórica")
biblio.agregar_libro("NOVE", "Cuentos de la Alhambra", "Washington Irving", "Histórica")
biblio.agregar_libro("NOVE", "Lisianca", "Sienkiewicz", "Histórica")
biblio.agregar_libro("NOVE", "Romancero del mío Cid", "Ed. Salvat", "Histórica")
biblio.agregar_libro("NOVE", "Trafalgar", "Benito Pérez Galdós", "Histórica")
biblio.agregar_libro("NOVE", "La letra escarlata", "Hawthorne", "Histórica")
biblio.agregar_libro("NOVE", "El médico de Toledo", "Toledano", "Histórica")
biblio.agregar_libro("NOVE", "El ruido del tiempo", "Julian Barnes", "Histórica")

biblio.agregar_libro("NOVE", "Los hermanos Karamazov", "Dostoyevski", "Realismo Social")
biblio.agregar_libro("NOVE", "Rastro de sándalo", "Asha Miró", "Realismo Social")
biblio.agregar_libro("NOVE", "El extranjero", "Albert Camus", "Realismo Social")
biblio.agregar_libro("NOVE", "Los años difíciles", "Alejandro Gándara", "Realismo Social")
biblio.agregar_libro("NOVE", "Todos los clásicos rusos", "Henry Luque Muñoz", "Realismo Social")
biblio.agregar_libro("NOVE", "Dublineses", "James Joyce", "Realismo Social")
biblio.agregar_libro("NOVE", "La buena tierra", "Pearl S. Buck", "Realismo Social")
biblio.agregar_libro("NOVE", "El pequeño burgués", "Balzac", "Realismo Social")
biblio.agregar_libro("NOVE", "Tener y no tener", "Hemingway", "Realismo Social")
biblio.agregar_libro("NOVE", "La madre", "Gorki", "Realismo Social")
biblio.agregar_libro("NOVE", "Los luchadores", "Pío Baroja", "Realismo Social")
biblio.agregar_libro("NOVE", "Tiempo de silencio", "Luis Martín-Santos", "Realismo Social")
biblio.agregar_libro("NOVE", "La isla", "Mara Torres", "Realismo Social")
biblio.agregar_libro("NOVE", "La tregua", "Mario Benedetti", "Realismo Social")
biblio.agregar_libro("NOVE", "Así se templó el acero", "Nikolai Ostrovski", "Realismo Social")
biblio.agregar_libro("NOVE", "The Crack-Up", "F. Scott Fitzgerald", "Realismo Social")
biblio.agregar_libro("NOVE", "Diario II", "Anaïs Nin", "Realismo Social")
biblio.agregar_libro("NOVE", "Un pueblecito", "Azorín", "Realismo Social")
biblio.agregar_libro("NOVE", "La novela de un novelista", "Armando Palacio Valdés", "Realismo Social")
biblio.agregar_libro("NOVE", "La colmena", "Cela", "Realismo Social")
biblio.agregar_libro("NOVE", "La princesa de Cléveris", "Madame de La Fayette", "Realismo Social")
biblio.agregar_libro("NOVE", "Algunos muchachos y otros cuentos", "Ana María Matute", "Realismo Social")
biblio.agregar_libro("NOVE", "Narraciones", "Chéjov", "Realismo Social")
biblio.agregar_libro("NOVE", "Camisa roja", "Miguel Delibes", "Realismo Social")
biblio.agregar_libro("NOVE", "Café de artistas y otros cuentos", "Cela", "Realismo Social")
biblio.agregar_libro("NOVE", "Parábola", "Juan José Dos Santos", "Realismo Social")
biblio.agregar_libro("NOVE", "De lo poco de la vida de hoya", "José Luis de Tomás García", "Realismo Social")
biblio.agregar_libro("NOVE", "El jardinero", "Tagore", "Realismo Social")
biblio.agregar_libro("NOVE", "La tribuna", "Emilia Pardo Bazán", "Realismo Social")
biblio.agregar_libro("NOVE", "Una desdichada", "Turgeneiev", "Realismo Social")
biblio.agregar_libro("NOVE", "El último magnate", "F. Scott Fitzgerald", "Realismo Social")
biblio.agregar_libro("NOVE", "Balzac y la comedia humana", "Carlos Pujol", "Realismo Social")
biblio.agregar_libro("NOVE", "Cuentos escogidos", "Andreiev", "Realismo Social")
biblio.agregar_libro("NOVE", "El guardián entre el centeno", "J. D. Salinger", "Realismo Social")
biblio.agregar_libro("NOVE", "Prisioneros del agua", "Alexis Díaz-Pimienta", "Realismo Social")
biblio.agregar_libro("NOVE", "Tiempo de silencio", "Luis Martín-Santos", "Realismo Social")
biblio.agregar_libro("NOVE", "Mi siglo", "Günter Grass", "Realismo Social")
biblio.agregar_libro("NOVE", "La madre", "Pearl S. Buck", "Realismo Social")
biblio.agregar_libro("NOVE", "Los ex hombres", "Gorki", "Realismo Social")
biblio.agregar_libro("NOVE", "Viento del este, viento del oeste", "Pearl S. Buck", "Realismo Social")
biblio.agregar_libro("NOVE", "Los señores Golovliov", "Mijaíl Saltykov-Schedrin", "Realismo Social")
biblio.agregar_libro("NOVE", "El camino", "Miguel Delibes", "Realismo Social")
biblio.agregar_libro("NOVE", "El príncipe de Maquiavelo", "Nicolás Maquiavelo", "Realismo Social")
biblio.agregar_libro("NOVE", "El cortesano", "Baldasar Castiglione", "Realismo Social")
biblio.agregar_libro("NOVE", "El jugador", "Dostoyevski", "Realismo Social")
biblio.agregar_libro("NOVE", "Cuentos rusos", "Ed. Salvat", "Realismo Social")
biblio.agregar_libro("NOVE", "Nido de hidalgos y vísperas", "Turguenev", "Realismo Social")
biblio.agregar_libro("NOVE", "La Regenta", "Leopoldo Alas \"Clarín\"", "Realismo Social")
biblio.agregar_libro("NOVE", "Senderos de la libertad", "Javier Moro", "Realismo Social")

biblio.agregar_libro("NOVE", "La pasión turca", "Antonio Gala", "Romántica")
biblio.agregar_libro("NOVE", "Así empezó todo", "Mary Nickson", "Romántica")
biblio.agregar_libro("NOVE", "Donde el corazón te lleve", "Susanna Tamaro", "Romántica")
biblio.agregar_libro("NOVE", "La sonata a Kreutzer", "Tolstoi", "Romántica")
biblio.agregar_libro("NOVE", "Una noche encantada", "Christina Dodd", "Romántica")
biblio.agregar_libro("NOVE", "El señor de la guerra", "Elizabeth Elliott", "Romántica")
biblio.agregar_libro("NOVE", "Al otro lado del río y entre los árboles", "Ernest Hemingway", "Romántica")
biblio.agregar_libro("NOVE", "El imposible olvido", "Antonio Gala", "Romántica")
biblio.agregar_libro("NOVE", "Orgullo y prejuicio", "Jane Austen", "Romántica")
biblio.agregar_libro("NOVE", "La tregua", "Mario Benedetti", "Romántica")
biblio.agregar_libro("NOVE", "Werther", "Goethe", "Romántica")
biblio.agregar_libro("NOVE", "Secretos en la provincia", "Bridget Asher", "Romántica")
biblio.agregar_libro("NOVE", "A este lado del paraíso", "F. Scott Fitzgerald", "Romántica")
biblio.agregar_libro("NOVE", "Al este del Edén", "John Steinbeck", "Romántica")
biblio.agregar_libro("NOVE", "Desayuno en Tiffany's", "Truman Capote", "Romántica")
biblio.agregar_libro("NOVE", "Mujeres enamoradas", "D. H. Lawrence", "Romántica")
biblio.agregar_libro("NOVE", "El amante de Lady Chatterley", "D. H. Lawrence", "Romántica")
biblio.agregar_libro("NOVE", "La sonrisa etrusca", "José Luis Sampedro", "Romántica")
biblio.agregar_libro("NOVE", "Prado Florido", "John Galsworthy", "Romántica")
biblio.agregar_libro("NOVE", "La insoportable levedad del ser", "Milan Kundera", "Romántica")
biblio.agregar_libro("NOVE", "El caballero Mauprat", "Jorge Sandi", "Romántica")
biblio.agregar_libro("NOVE", "La dama de las camelias", "Dumas hijo", "Romántica")
biblio.agregar_libro("NOVE", "Don Juan", "Lord Byron", "Romántica")

biblio.agregar_libro("NOVE", "El Palacio de Medianoche", "Carlos Ruiz Zafón", "Suspense")
biblio.agregar_libro("NOVE", "La Sombra del Viento", "Carlos Ruiz Zafón", "Suspense")
biblio.agregar_libro("NOVE", "El Prisionero del Cielo", "Carlos Ruiz Zafón", "Suspense")
biblio.agregar_libro("NOVE", "Marina", "Carlos Ruiz Zafón", "Suspense")
biblio.agregar_libro("NOVE", "Ocho mortales poseídas", "Tennessee Williams", "Suspense")
biblio.agregar_libro("NOVE", "Extraños en un tren", "Patricia Highsmith", "Suspense")
biblio.agregar_libro("NOVE", "Horizonte de sucesos", "Carmen Peire", "Suspense")
biblio.agregar_libro("NOVE", "El código Da Vinci", "Dan Brown", "Suspense")
biblio.agregar_libro("NOVE", "La coartada perfecta", "Patricia Highsmith", "Suspense")
biblio.agregar_libro("NOVE", "Nuestro juego", "John Le carré", "Suspense")
biblio.agregar_libro("NOVE", "El caso del crimen de la ópera", "Elia Barceló", "Suspense")
biblio.agregar_libro("NOVE", "Sherlock Holmes: Estrella de plata", "Arthur Conan Doyle", "Suspense")
biblio.agregar_libro("NOVE", "Daddy", "Loup Durand", "Suspense")
biblio.agregar_libro("NOVE", "El ocho", "Katherine Neville", "Suspense")
biblio.agregar_libro("NOVE", "Salvatore Giuliano, el siciliano", "Mario Puzo", "Suspense")
biblio.agregar_libro("NOVE", "El secuestro de Mamá y otros relatos del Marqués de Sotosancho", "Alfonso Díaz", "Suspense")
biblio.agregar_libro("NOVE", "La hoguera de las vanidades", "Tom Wolfe", "Suspense")
biblio.agregar_libro("NOVE", "La chica del tren", "Paula Hawkings", "Suspense")
biblio.agregar_libro("NOVE", "El fuego invisible", "Javier Sierra", "Suspense")
biblio.agregar_libro("NOVE", "El hiperboloide del ingeniero Garín", "Alekséi Tolstói", "Suspense")
biblio.agregar_libro("NOVE", "La mortaja", "Miguel Delibes", "Suspense")
biblio.agregar_libro("NOVE", "La muerte en Venecia", "Thomas Mann", "Suspense")
biblio.agregar_libro("NOVE", "El Fantasma de Canterville", "Oscar Wilde", "Suspense")
biblio.agregar_libro("NOVE", "Si, con la policía", "A. y G. Veimar", "Suspense")
biblio.agregar_libro("NOVE", "El balneario", "Martin Gaite", "Suspense")
biblio.agregar_libro("NOVE", "El perseguidor", "Cortázar", "Suspense")
biblio.agregar_libro("NOVE", "El coronel no tiene quien le escriba", "Gabriel García Márquez", "Suspense")
biblio.agregar_libro("NOVE", "Una pequeña ciudad de Alemania", "John Le Carré", "Suspense")
biblio.agregar_libro("NOVE", "Crónica de una muerte anunciada", "Gabriel García Márquez", "Suspense")
biblio.agregar_libro("NOVE", "El susto de Mr. Ripley", "Patricia Highsmith", "Suspense")
biblio.agregar_libro("NOVE", "El ameriacano imposible", "Graham Greene", "Suspense")
biblio.agregar_libro("NOVE", "Los amigos del crimen perfecto", "Andrés Trapiello", "Suspense")
biblio.agregar_libro("NOVE", "Un espía perfecto", "John Le Carré", "Suspense")

# ==== RELIGIÓN ====
biblio.agregar_libro("RELI", "El inconsciente, ¿morada de Dios?", "Josep Otón Catalán")
biblio.agregar_libro("RELI", "Más allá de la muerte", "Hélène Renard")
biblio.agregar_libro("RELI", "Consagración al Sagrado Corazón de Jesús", "Iglesia de Mataelpino")
biblio.agregar_libro("RELI", "Sinopsis de los cuatro evangelios", "Juan Leal")
biblio.agregar_libro("RELI", "Ramakrishna y la vitalidad del hinduismo", "Solange Lemaitre")
biblio.agregar_libro("RELI", "Culturas y religiones en diálogo", "Juan José Tamayo y María José Fariñas")
biblio.agregar_libro("RELI", "El gran secreto de Jesús", "Juan Arias")
biblio.agregar_libro("RELI", "Una lectura social del Nuevo Testamento", "Fidel Aizpurúa Donazar")
biblio.agregar_libro("RELI", "Asia menor en tiempos de Pablo, Lucas y Juan", "Eduardo Arens")
biblio.agregar_libro("RELI", "La sabiduría de los antepasados", "T. Lobsang Rampa")
biblio.agregar_libro("RELI", "La religión en la cultura primitiva", "Edward Burnett Tylor")
biblio.agregar_libro("RELI", "El poder de la magia sagrada", "Anónimo")
biblio.agregar_libro("RELI", "El conflicto de los siglos", "Elena G. White")
biblio.agregar_libro("RELI", "El poder desconocido del ayuno", "Sor Emmanuel Maillard")
biblio.agregar_libro("RELI", "Breviarium romanum ex decreto Sacrosancti Concilij Tridentini restitutum", "Pío V y Clemente VIII")
biblio.agregar_libro("RELI", "El Corán", "Mahoma")
biblio.agregar_libro("RELI", "Biblia Sacra Vulgata", "Editorial Weber - Gryson")
biblio.agregar_libro("RELI", "La Santa Biblia", "San Pablo")
biblio.agregar_libro("RELI", "Mi fe en la verdad", "Mohan Das Gandhi")
biblio.agregar_libro("RELI", "Los sabios de la túnica color ciruela", "Fun-Chang")
biblio.agregar_libro("RELI", "Evangelio según Tomás", "Ediciones Obelisco")
biblio.agregar_libro("RELI", "Tao Te King", "Lao Tse")
biblio.agregar_libro("RELI", "Selección de cantos religiosos populares", "P. Pliama Editorial")
biblio.agregar_libro("RELI", "Los libros de Enoc", "Oliver Ibáñez")
biblio.agregar_libro("RELI", "Bibiani", "No aplicable")
biblio.agregar_libro("RELI", "Sagrada Biblia", "Ediciones S.A.")
biblio.agregar_libro("RELI", "La Madonna e il bambino Gesú", "Revista Gente")
biblio.agregar_libro("RELI", "El libro verde", "Mammar El Ghadafi")
biblio.agregar_libro("RELI", "El libro de los muertos", "Anónimo")
biblio.agregar_libro("RELI", "La caverna de los antepasados", "T. Lobsang Rampa")


El libro 'El arte del estudio de Ajedrez' de Zoilo R. Caputto ya existe en la sección Ajedrez.
El libro 'El gran libro de ajedrez' de Natale Ramini ya existe en la sección Ajedrez.
El libro 'Larousse del Ajedrez' de Larousse ya existe en la sección Ajedrez.
El libro 'Ajedrez Español' de Editorial SEAS ya existe en la sección Ajedrez.
El libro 'Las batallas de las ideas en el Ajedrez' de Anthony Saidy ya existe en la sección Ajedrez.
El libro 'Ajedrez, maestro contra Amateur' de M. Euwe y W. Meiden ya existe en la sección Ajedrez.
El libro 'El arte del Sacrificio en el ajedrez' de Editorial de Vecchi ya existe en la sección Ajedrez.
El libro 'The Ultimate Pirc' de John Nunn ya existe en la sección Ajedrez.
El libro 'Premios de belleza en el Ajedrez' de François Le Lionnais ya existe en la sección Ajedrez.
El libro 'Táctica moderna en el Ajedrez' de Ludek Pachmann ya existe en la sección Ajedrez.
El libro 'Aperturas Semiabiertas I' de Ludek Pachmann ya existe en la sección Ajedrez.
El li

'RELI.30'

### Seccion ### Subsección ###  Autor ### Titulo ###

In [14]:
# Todos los libros de una sección
#biblio.eliminar_libro("FILO","Isagoge","Porfirio");

In [15]:
#biblio.buscar_libros(titulo="biblia");

In [23]:
biblio.buscar_libros(subseccion="Bélica");

NOVE.BÉLI.1 - Mar fuerte que el último (Gueorgui Bernátzko)
NOVE.BÉLI.2 - Tiempo de prueba (Pavel Nilov)
NOVE.BÉLI.3 - De los vivos y de los muertos (Konstantin Simonov)
NOVE.BÉLI.4 - Balada alpina (Vasil Bykov)
NOVE.BÉLI.5 - Un día en la vida de Ivan Denisovich (Aleksandr Solzhenitsyn)
NOVE.BÉLI.6 - Un hombre de verdad (Boris Polevoy)
NOVE.BÉLI.7 - La joven guardia (Alexander Fadéiev)


In [17]:
#Eliminar libros
#biblio.eliminar_libro("CIEN", "Fauna Ibérica, El hombre y la Tierra 1", "Félix Rodríguez de la Fuente", "Fauna")
#biblio.agregar_libro("FILO", "Einstein. Historia de un espíritu", "Desiderio Papp")
#biblio.agregar_libro("CIEN", "Fauna Ibérica, El hombre y la Tierra X", "Félix Rodríguez de la Fuente", "Fauna")


In [18]:
#print(json.dumps(biblio.catalogo, indent=4, ensure_ascii=False))

In [22]:
libros_totales = biblio.buscar_libros()
print(f"\nActualmente la Biblioteca de Ana tiene {len(libros_totales)} libros")

AJED.1 - El arte del estudio de Ajedrez (Zoilo R. Caputto)
AJED.2 - El gran libro de ajedrez (Natale Ramini)
AJED.3 - Larousse del Ajedrez (Larousse)
AJED.4 - Ajedrez Español (Editorial SEAS)
AJED.5 - Las batallas de las ideas en el Ajedrez (Anthony Saidy)
AJED.6 - Ajedrez, maestro contra Amateur (M. Euwe y W. Meiden)
AJED.7 - El arte del Sacrificio en el ajedrez (Editorial de Vecchi)
AJED.8 - The Ultimate Pirc (John Nunn)
AJED.9 - Premios de belleza en el Ajedrez (François Le Lionnais)
AJED.10 - Táctica moderna en el Ajedrez (Ludek Pachmann)
AJED.11 - Aperturas Semiabiertas I (Ludek Pachmann)
AJED.12 - Aperturas Abiertas (Ludek Pachmann)
AJED.13 - Táctica moderna en el Ajedrez (repetido) (Ludek Pachmann)
AJED.14 - Aperturas Semiabiertas II (Ludek Pachmann)
AJED.15 - Garry Kasparov (M. Yudovich)
AJED.16 - El Ajedrez (Ricardo Aguilera)
AJED.17 - Corona Mundial de Ajedrez Kasparov-Karpov (Inaplicable)
AJED.18 - Los principios del Ajedrez (Ray Arvid)
AJED.19 - La partida de Ajedrez (Rubin

In [24]:
biblio.guardar("biblioteca.json")
